In [1]:
import rdflib
import re
import numpy as np
from SPARQLWrapper import SPARQLWrapper, JSON
from pyjarowinkler import distance
import logging
from six import iteritems

In [2]:
%%time
glove_vectors_file = "C:\\Users\\daix8\\Desktop\\CSI5386\\FinalProj\\glove.6B.300d.txt"
glove_wordmap = {}
with open(glove_vectors_file, "r", encoding="utf-8") as glove:
    for line in glove:
        name, vector = tuple(line.split(" ", 1))
        glove_wordmap[name] = np.fromstring(vector, sep=" ")

Wall time: 51.7 s


In [3]:
#sparql = SPARQLWrapper("http://134.117.101.79:8890/sparql/")
sparql = SPARQLWrapper("http://dbpedia.org/sparql")

In [4]:
def evaluate_similarity_score(wemb, ar):
    res = []
    for x in ar:
        if x[0] in wemb and x[1] in wemb:
            vector1 = wemb[x[0]]
            vector2 = wemb[x[1]]  
            res.append(np.dot(vector1,vector2)/(np.linalg.norm(vector1)*(np.linalg.norm(vector2))))
        else:
            res.append(-1)
    return res

In [5]:
import textdistance

def levenshteinSim(w1,w2):
    m = max(len(w1),len(w2))
    return (m-textdistance.levenshtein(w1,w2))/m


In [6]:
import nltk
import spacy
nlp = spacy.load("en_core_web_sm")
from nltk.corpus import wordnet as wn
import textdistance

def similarity(w1,w2):
    synsets1 = wn.synsets(w1)
    synsets2 = wn.synsets(w2)

    m = max(len(w1),len(w2))
    levDis = (m-textdistance.levenshtein(w1,w2))/m

    maxScore = 0
    maxWup = 0
    maxLch = 0
    for s1 in synsets1:
        for s2 in synsets2:

            #Path similarity
            currentPath = wn.path_similarity(s1,s2)
            if currentPath != None and maxScore < currentPath:
                maxScore = currentPath

            #Wu-Palmer similarity
            currentWup = wn.wup_similarity(s1,s2)
            if currentWup != None and maxWup < currentWup:
                maxWup = currentWup

            #Leacock-Chodorow Similarity, need same part of speech for both words
            Max_Lch = 3.6375
            try:
                currentLch = wn.lch_similarity(s1,s2) / Max_Lch
            except:
                currentLch = 0
            if currentLch != None and maxLch < currentLch:
                maxLch = currentLch
    '''
    print(maxScore)
    print(maxWup)
    print(maxLch)
    print(levDis)
    '''
    ans = [maxScore,maxWup,maxLch,levDis]
    if ans.count(0) == len(ans):
        return 0
    return sum(ans) / (len(ans) - ans.count(0))
    

In [7]:
import re
def splitSimilarity(w1,w2):
    maxScore = 0

    w1split = re.sub( r"([A-Z])", r" \1", w1).split()
    w2split = re.sub( r"([A-Z])", r" \1", w2).split()

    for x in w1split:
        for y in w2split:
            currentScore = similarity(x,y)
            #currentScore = calculateCost(x,y)
            if currentScore > maxScore:
                maxScore = currentScore
    
    #print(w1split)
    #print(w2split)
    return '{0:.4f}'.format(maxScore)

In [8]:
def createQueryString(label):
    th = 10000
    if countIncomingEdges(label) < th:
    #query = 'select distinct ?s ?p ?j ?k where {{?s ?p <' + label +'>} UNION {<' + label +'> ?j ?k}}'
    #filter wiki
        query = 'select distinct ?s ?p ?j ?k where {{?s ?p <' + label +'>. filter(!regex(str(?p), "wikiPage") && !regex(str(?p), "rdf-syntax"))} UNION {<' + label +'> ?j ?k. filter(!regex(str(?j), "wikiPage") && !regex(str(?j), "rdf-syntax"))}}'
    else:
        query = 'select distinct ?j ?k where {<' + label +'> ?j ?k. filter(!regex(str(?j), "wikiPage") && !regex(str(?j), "rdf-syntax"))}'
    return query

In [9]:
def countIncomingEdges(obj):
    queryString = "SELECT count(DISTINCT ?s) as ?c WHERE { ?s ?p <" + obj +">}"
    sparql.setQuery(queryString)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    #print(queryString)
    for result in results["results"]["bindings"]:
        return int(result['c']['value'])
countIncomingEdges("http://dbpedia.org/resource/Elton_John")

4990

In [10]:
class TRIPLE:
    def __init__(self,s,p,o,seed):
        self.subject = s
        self.predicate = p
        self.object = o
        self.seeds = seed
        self.cost = 0
        self.prob = 0
        self.previousTriple = None
        self.matchedPredicates = []
        self.matchedDataPredicates = []
    def __str__(self):
        #return self.subject + '  ---  ' + self.predicate +'  ---  ' + self.object + "("+ str(self.prob) + ")"
        return str(self.matchedPredicates) + str(self.matchedDataPredicates) + '\\' + self.subject + '  ---  ' + self.predicate +'  ---  ' + self.object + "("+ str(self.cost) + ', '+ str(self.prob) +', '+ str(len(self.matchedPredicates))+ ")" + ' ||||| ' + str(self.previousTriple)
    def getObj(self):
        return self.object
    def getPredicate(self):
        return self.predicate
    def getSeeds(self):
        return self.seeds
    def insertCost(self,keywords):
        ans = []
        pre = self.predicate[self.predicate.rindex('/')+1:]
        
        currentMax = 0
        currentKeyword = 'None'
        matchedKeywordThreshold = 0.6
        
        for kw in keywords:
            #ans.append(splitSimilarity(kw,pre))
            #ans.append(similarity(kw,pre))
            #ans.append(calculateCost2(kw,pre))
            
            if kw not in self.matchedPredicates:
                currentScore = float('{:.2f}'.format(calculateCost2(kw,pre)))
                if currentScore > currentMax:
                    currentMax = currentScore
                    currentKeyword = kw
        
        if currentKeyword not in self.matchedPredicates and currentScore > matchedKeywordThreshold:
            self.matchedPredicates.append(currentKeyword)
        
        currentCost = currentMax
        self.cost = currentCost
        
        if self.previousTriple is None:
            self.prob = currentCost
            
        else:
            self.prob = currentCost * self.previousTriple.getProb()
    def getCost(self):
        return self.cost
    
    def __gt__(self,other):
        
        if len(self.matchedPredicates) < len(other.getMatchedPredicates()):
            return True
        
        if self.prob < other.getProb():
            return True
        else:
            return False
    
    def getProb(self):
        return self.prob
    
    def getMatchedPredicates(self):
        return self.matchedPredicates[:]
    
    def getMatchedDataPredicates(self):
        return self.matchedDataPredicates[:]
    
    def setPreviousTriple(self,tp):
        self.previousTriple = tp
        self.matchedPredicates = tp.getMatchedPredicates()
        self.matchedDataPredicates = tp.getMatchedDataPredicates()
    
    

In [11]:
class TRIPLE:
    def __init__(self,s,p,o,seed):
        self.subject = s
        self.predicate = p
        self.object = o
        self.seeds = seed
        self.cost = 0
        self.prob = 0
        self.previousTriple = None
        self.matchedPredicates = []
        self.matchedDataPredicates = []
    def __str__(self):
        return self.subject + '  ---  ' + self.predicate +'  ---  ' + self.object + "("+ str(self.prob) + ")"
        #return str(self.matchedPredicates) + str(self.matchedDataPredicates) + '\\' + self.subject + '  ---  ' + self.predicate +'  ---  ' + self.object + "("+ str(self.cost) + ', '+ str(self.prob) +', '+ str(len(self.matchedPredicates))+ ")" + ' ||||| ' + str(self.previousTriple)
    def getObj(self):
        return self.object
    def getPredicate(self):
        return self.predicate
    def getSeeds(self):
        return self.seeds
    def insertCost(self,keywords):
        ans = []
        pre = self.predicate[self.predicate.rindex('/')+1:]
        
        currentMax = 0
        currentKeyword = 'None'
        currentPre = 'None'
        matchedKeywordThreshold = 0.65
        
        for kw in keywords:
            #ans.append(splitSimilarity(kw,pre))
            #ans.append(similarity(kw,pre))
            #ans.append(calculateCost2(kw,pre))
            
            if kw not in self.matchedPredicates and pre not in self.matchedDataPredicates:
                currentScore = float('{:.2f}'.format(calculateCost2(kw,pre)))
                if currentScore > currentMax:
                    currentMax = currentScore
                    currentKeyword = kw
                    currentPre = pre
        
        if currentKeyword not in self.matchedPredicates and currentMax > matchedKeywordThreshold:
            self.matchedPredicates.append(currentKeyword)
            self.matchedDataPredicates.append(currentPre)
        
        currentCost = currentMax
        self.cost = currentCost
        
        if self.previousTriple is None:
            self.prob = currentCost
            
        else:
            self.prob = currentCost * self.previousTriple.getProb()
    def getCost(self):
        return self.cost
    
    def __gt__(self,other):
        
        if len(self.matchedPredicates) < len(other.getMatchedPredicates()):
            return True
        elif len(self.matchedPredicates) == len(other.getMatchedPredicates()):
            if self.prob < other.getProb():
                return True
            else:
                return False
        else:
            return False
    
    def getProb(self):
        return self.prob
    
    def getMatchedPredicates(self):
        return self.matchedPredicates[:]
    
    def getMatchedDataPredicates(self):
        return self.matchedDataPredicates[:]
    
    def setPreviousTriple(self,tp):
        self.previousTriple = tp
        self.matchedPredicates = tp.getMatchedPredicates()
        self.matchedDataPredicates = tp.getMatchedDataPredicates()
    
    

In [12]:
def checkDup(eg, triple):
    dup = set()
    for tp in eg:
        if tp.getObj() == triple.getObj() and tp.getSeeds() != triple.getSeeds():
            dup.add(tp)
    return dup

In [13]:
import spacy
nlp = spacy.load("en_core_web_sm")

def calculateCost(w1,w2):
    w1lemma = nlp(w1)[0].lemma_
    w2lemma = nlp(w2)[0].lemma_
    
    pair = [[w1lemma,w2lemma]]
    wescore = evaluate_similarity_score(glove_wordmap, pair)[0]
    if wescore != -1:
        return wescore
    else:
        return levenshteinSim(w1,w2)

In [14]:
def calculateCost2(w1,w2):
    WEWeight = 0.75
    
    w1lemma = nlp(w1)[0].lemma_
    w2lemma = nlp(w2)[0].lemma_
    
    pair = [[w1lemma,w2lemma]]
    wescore = evaluate_similarity_score(glove_wordmap, pair)[0]
    
    if wescore == -1:
        return similarity(w1,w2)
    else:
        return wescore * WEWeight + similarity(w1,w2) * (1 - WEWeight)
    
    # without weights
    '''
    if wescore != -1:
        return wescore
    else:
        return similarity(w1,w2)
    '''

In [15]:
def fetchTopK(li,k):
    ans = []
    current = set()
    c = 0
    while c<len(li) and len(current) < k+1:
        #print(len(current))
        pstring = li[c].getPredicate()
        pre = pstring[pstring.rindex('/')+1:]
        if pre in current:
            ans.append(li[c])
        elif pre not in current:
            current.add(pre)
            ans.append(li[c])
        c += 1
    if len(current) == k + 1:    
        ans.pop(-1)
    return ans

In [16]:
with open('215ques_correct.txt','r') as f:
    lines = f.readlines()
c = 0
lbs = []
current = []
while c < len(lines):
    line = lines[c].strip()
    if line.isnumeric():
        lbs.append(current)
        current = []
    else:
        current.append(line.strip())
    c += 1
lbs.pop(0)
lbs.append(['http://dbpedia.org/resource/Sparkling_wines','http://dbpedia.org/resource/France'])

#read keywords: original -> resultsKeywords.txt  Answer-> kwanswer.txt
with open('./newKeywordsList/onlykw.txt','r') as f:
#with open('resultsKeywords.txt','r') as f:
    lines = f.readlines()
    
keywordsList = []
for x in lines:
    line = x.strip().split(',')
    i = 0
    while i < len(line):
        line[i] = line[i].strip()
        i += 1
    keywordsList.append(line)

In [17]:
with open('./newKeywordsList/Type.txt','r',encoding='utf-8') as pf:
    pflines = pf.readlines()
pdic = {}
for line in pflines:
    li = line.strip().split(',')
    p = li[0]
    ct = li[1]
    pdic[p] = ct

def getAnswerType(kw, pd):
    th = 4000
    
    maxCount = th
    maxType = 'None'
    
    for k in kw:
        for p in pdic:
            if k.lower() == p.lower() and maxCount < int(pdic[p]):
                maxType = p
    
    return maxType

In [18]:
def filterTypeQuery(ans, ent):
    if not ans.startswith('http://dbpedia.org/resource/'):
        return False
    
    query = 'ASK WHERE{<'+ ans + '> rdf:type <http://dbpedia.org/ontology/' + ent +'>} '
    
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    
    return results['boolean']
filterTypeQuery('http://dbpedia.org/resource/On_the_Road','Book')

True

# The case when number of entities >= 3

In [20]:
class resultGraph:
    def __init__(self):
        self.triples = []
        self.matchedKeywords = []
        self.prob = 1
        self.ans = None
    
    def insertTriple(self,tp):
        self.triples.append(tp)
    
    def __str__(self):
        res = ''
        for x in self.triples:
            res += str(x) + '\n'
        return res
    
    def setAnsTriple(self, tp):
        self.ans = tp
    
    def getAnsTriple(self):
        return self.ans
    
    def computeCost(self, keywords, func):
        
        #threshold for matched predicate
        th = 0.4
        
        for x in self.triples:
            currentMax = 0
            currentKeyword = None
            
            originalPre = x.getPredicate()
            pre = originalPre[originalPre.rindex('/')+1:]
            
            
            for y in keywords:
                if y in self.matchedKeywords:
                    continue
                    
                currentScore = func(y, pre)
                if currentScore > currentMax:
                    currentMax = currentScore
                    currentKeyword = y
            
            if currentMax > th:
                x.setCost(currentMax)
                self.matchedKeywords.append(currentKeyword)
                x.setMatchedKeyword(currentKeyword)
    
        self.computeProb()
        
    def computeProb(self):
        for x in self.triples:
            self.prob *= x.getCost()
    
    def getProb(self):
        return self.prob
    
    def getNumMatchedKeywords(self):
        return len(self.matchedKeywords)
    
    def __lt__(self,other):
        if self.getNumMatchedKeywords() > other.getNumMatchedKeywords():
            return True
        elif self.getNumMatchedKeywords() < other.getNumMatchedKeywords():
            return False
        else:
            if self.getProb() > other.getProb():
                return True
            else:
                return False
    
class TripleV2:
    def __init__(self,s,p,o):
        self.subject = s
        self.predicate = p
        self.object = o
        self.cost = 1
        self.matchtedKeyword = None
    
    def __str__(self):
        return self.subject + '  ---  ' + self.predicate +'  ---  ' + self.object + '(' + str(self.cost) + ' : ' + str(self.matchtedKeyword) +')'

    def getPredicate(self):
        return self.predicate
    
    def setCost(self, c):
        self.cost = c
        
    def getCost(self):
        return self.cost
    
    def setMatchedKeyword(self,k):
        self.matchtedKeyword = k
        
r1 = resultGraph()
r1.insertTriple(TripleV2('sub1', 'http://dbpedia.org/ontology/spouse','obj1'))
r1.insertTriple(TripleV2('sub2', 'http://dbpedia.org/ontology/parent','obj2'))
r1.insertTriple(TripleV2('sub3', 'http://dbpedia.org/ontology/city','obj3'))
r1.computeCost(['wife','parents','test'], calculateCost2)
print(r1)

r2 = resultGraph()
r2.insertTriple(TripleV2('sub1', 'http://dbpedia.org/ontology/spouse','obj1'))
r2.insertTriple(TripleV2('sub2', 'http://dbpedia.org/ontology/parent','obj2'))
r2.insertTriple(TripleV2('sub3', 'http://dbpedia.org/ontology/exam','obj3'))
r2.computeCost(['wife','parents','test'], calculateCost2)
print(r2)

print(r1>r2)

sub1  ---  http://dbpedia.org/ontology/spouse  ---  obj1(0.525735826685831 : wife)
sub2  ---  http://dbpedia.org/ontology/parent  ---  obj2(0.9910729089790984 : parents)
sub3  ---  http://dbpedia.org/ontology/city  ---  obj3(1 : None)

sub1  ---  http://dbpedia.org/ontology/spouse  ---  obj1(0.525735826685831 : wife)
sub2  ---  http://dbpedia.org/ontology/parent  ---  obj2(0.9910729089790984 : parents)
sub3  ---  http://dbpedia.org/ontology/exam  ---  obj3(0.5656071759498349 : test)

True


In [22]:
%%time
def threeEntityTemp():
    e1 = 'http://dbpedia.org/resource/Viking_Press'
    e2 = 'http://dbpedia.org/resource/Jack_Kerouac'
    e3 = 'http://dbpedia.org/resource/Beat_Generation'
    
    keywords = ['written','published','literaturalGenre']
    res = []
    
    #print('--- h1 ---')
    for x in helper1(e1,e2,e3):
        res.append(x)
    
    #print('--- h2 ---')
    for x in helper2(e1,e2,e3):
        res.append(x)
    for x in helper2(e2,e1,e3):
        res.append(x)
    for x in helper2(e3,e1,e2):
        res.append(x)
    
    #print('--- h3 ---')
    for x in helper3(e1,e2,e3):
        res.append(x)
    for x in helper3(e2,e1,e3):
        res.append(x)
    for x in helper3(e3,e1,e2):
        res.append(x)
    
    
    #print('--- h4 ---')
    for x in helper4(e1,e2,e3):
        res.append(x)
    
    for x in helper5(e1,e2,e3):
        res.append(x)
    for x in helper5(e1,e3,e2):
        res.append(x)
    for x in helper5(e2,e3,e1):
        res.append(x)
        
    for x in helper6(e1,e2,e3):
        res.append(x)
    for x in helper6(e1,e3,e2):
        res.append(x)
    for x in helper6(e2,e3,e1):
        res.append(x)
    
    for x in helper7(e1,e2,e3):
        res.append(x)
    for x in helper7(e1,e3,e2):
        res.append(x)
    for x in helper7(e2,e3,e1):
        res.append(x)
    
    
    for x in helper8(e1,e2,e3):
        res.append(x)
    for x in helper8(e1,e3,e2):
        res.append(x)
    for x in helper8(e2,e3,e1):
        res.append(x)
    
    for x in helper9(e1,e2,e3):
        res.append(x)
    for x in helper9(e1,e3,e2):
        res.append(x)
    for x in helper9(e2,e1,e3):
        res.append(x)
    for x in helper9(e2,e3,e1):
        res.append(x)
    for x in helper9(e3,e2,e1):
        res.append(x)
    for x in helper9(e3,e1,e2):
        res.append(x)
    
    for x in helper10(e1,e2,e3):
        res.append(x)
    for x in helper10(e1,e3,e2):
        res.append(x)
    for x in helper10(e2,e1,e3):
        res.append(x)
    for x in helper10(e2,e3,e1):
        res.append(x)
    for x in helper10(e3,e2,e1):
        res.append(x)
    for x in helper10(e3,e1,e2):
        res.append(x)
    
    
    for x in helper11(e1,e2,e3):
        res.append(x)
    for x in helper11(e1,e3,e2):
        res.append(x)  
    for x in helper11(e2,e3,e1):
        res.append(x)
    
    
    for x in helper12(e1,e2,e3):
        res.append(x)
    for x in helper12(e1,e3,e2):
        res.append(x)  
    for x in helper12(e2,e3,e1):
        res.append(x)

    for x in helper13(e1,e2,e3):
        res.append(x)
    for x in helper13(e1,e3,e2):
        res.append(x)
    for x in helper13(e2,e1,e3):
        res.append(x)
    for x in helper13(e2,e3,e1):
        res.append(x)
    for x in helper13(e3,e2,e1):
        res.append(x)
    for x in helper13(e3,e1,e2):
        res.append(x)
    
    
    for x in helper14(e1,e2,e3):
        res.append(x)
    for x in helper14(e1,e3,e2):
        res.append(x)
    for x in helper14(e2,e1,e3):
        res.append(x)
    for x in helper14(e2,e3,e1):
        res.append(x)
    for x in helper14(e3,e2,e1):
        res.append(x)
    for x in helper14(e3,e1,e2):
        res.append(x)
    
    for x in res:
        x.computeCost(keywords, calculateCost2)
    res.sort()
    
    return res
    
def helper1(e1,e2,e3):
    ans = []
    '''
    e1 - p1 - ?ans
    e2 - p2 - ?ans
    e3 - p3 - ?ans
    '''
    queryString = "SELECT distinct ?ans ?p ?pp ?ppp WHERE {<" + e1 +"> ?p ?ans. <" + e2 + "> ?pp ?ans. <" + e3 + "> ?ppp ?ans. filter(!regex(str(?p), 'wikiPage') && !regex(str(?pp), 'wikiPage') && !regex(str(?ppp), 'wikiPage') && !regex(str(?p), '#type') && !regex(str(?pp), '#type') && !regex(str(?ppp), '#type'))}"
    #print(queryString)
    sparql.setQuery(queryString)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    #print(queryString)
    for result in results["results"]["bindings"]:
        res = resultGraph()
        ansTp = TripleV2(e1, result['p']['value'], result['ans']['value'])
        res.setAnsTriple(ansTp)
        res.insertTriple(ansTp)
        res.insertTriple(TripleV2(e2, result['pp']['value'], result['ans']['value']))
        res.insertTriple(TripleV2(e3, result['ppp']['value'], result['ans']['value']))
        ans.append(res)
    return ans

def helper2(e1,e2,e3):
    ans = []
    '''
    ?ans - p1 - e1
    e2 - p2 - ?ans
    e3 - p3 - ?ans
    '''
    queryString = "SELECT distinct ?ans ?p ?pp ?ppp WHERE {?ans ?p <" + e1 +">. <" + e2 + "> ?pp ?ans. <" + e3 + "> ?ppp ?ans. filter(!regex(str(?p), 'wikiPage') && !regex(str(?pp), 'wikiPage') && !regex(str(?ppp), 'wikiPage') && !regex(str(?p), '#type') && !regex(str(?pp), '#type') && !regex(str(?ppp), '#type'))}"
    #print(queryString)
    sparql.setQuery(queryString)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    #print(queryString)
    for result in results["results"]["bindings"]:
        res = resultGraph()
        res.insertTriple(TripleV2(result['ans']['value'], result['p']['value'], e1))
        
        ansTp = TripleV2(e2, result['pp']['value'], result['ans']['value'])
        res.insertTriple(ansTp)
        res.setAnsTriple(ansTp)
        
        res.insertTriple(TripleV2(e3, result['ppp']['value'], result['ans']['value']))
        ans.append(res)
    return ans

def helper3(e1,e2,e3):
    ans = []
    '''
    e1 - p1 - ?ans
    ?ans - p2 - e2
    ?ans - p3 - e3
    '''
    queryString = "SELECT distinct ?ans ?p ?pp ?ppp WHERE {<" + e1 +"> ?p ?ans. ?ans ?pp <" + e2 + ">. ?ans ?ppp <" + e3 + ">. filter(!regex(str(?p), 'wikiPage') && !regex(str(?pp), 'wikiPage') && !regex(str(?ppp), 'wikiPage') && !regex(str(?p), '#type') && !regex(str(?pp), '#type') && !regex(str(?ppp), '#type'))}"
    #print(queryString)
    sparql.setQuery(queryString)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    #print(queryString)
    for result in results["results"]["bindings"]:
        res = resultGraph()
        
        ansTp = TripleV2(e1, result['p']['value'], result['ans']['value'])
        res.setAnsTriple(ansTp)
        res.insertTriple(ansTp)
        
        res.insertTriple(TripleV2(result['ans']['value'], result['pp']['value'], e2))
        res.insertTriple(TripleV2(result['ans']['value'], result['ppp']['value'], e3))
        ans.append(res)
    return ans

def helper4(e1,e2,e3):
    ans = []
    '''
    ?ans - p1 - e1
    ?ans - p2 - e2
    ?ans - p3 - e3
    '''
    queryString = "SELECT distinct ?ans ?p ?pp ?ppp WHERE {?ans ?p <" + e1 +">. ?ans ?pp <" + e2 + ">. ?ans ?ppp <" + e3 + ">. filter(!regex(str(?p), 'wikiPage') && !regex(str(?pp), 'wikiPage') && !regex(str(?ppp), 'wikiPage') && !regex(str(?p), '#type') && !regex(str(?pp), '#type') && !regex(str(?ppp), '#type'))}"
    #print(queryString)
    sparql.setQuery(queryString)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    #print(queryString)
    for result in results["results"]["bindings"]:
        res = resultGraph()
        
        ansTp = TripleV2(e1, result['p']['value'], result['ans']['value'])
        res.setAnsTriple(ansTp)
        
        res.insertTriple(TripleV2(result['ans']['value'], result['p']['value'], e1))
        res.insertTriple(TripleV2(result['ans']['value'], result['pp']['value'], e2))
        res.insertTriple(TripleV2(result['ans']['value'], result['ppp']['value'], e3))
        ans.append(res)
    return ans

def helper5(e1,e2,e3):
    ans = []
    '''
    ?e1 - p1 - ?ans
    ?e2 - p2 - ?ans
    ?e3 - p3 - ?int
    ?int - p4 - ?ans
    '''
    queryString = "SELECT distinct ?ans ?p ?pp ?ppp ?pppp ?int WHERE {<" + e1 +"> ?p ?ans. <" + e2 + "> ?pp ?ans. <" + e3 + "> ?ppp ?int. ?int ?pppp ?ans filter(!regex(str(?p), 'wikiPage') && !regex(str(?pp), 'wikiPage') && !regex(str(?ppp), 'wikiPage') && !regex(str(?pppp), 'wikiPage') && !regex(str(?p), '#type') && !regex(str(?pp), '#type') && !regex(str(?ppp), '#type') && !regex(str(?pppp), '#type'))}"
    #print(queryString)
    sparql.setQuery(queryString)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    #print(queryString)
    for result in results["results"]["bindings"]:
        res = resultGraph()
        
        ansTp = TripleV2(e1, result['p']['value'], result['ans']['value'])
        res.setAnsTriple(ansTp)
        res.insertTriple(ansTp)
        
        res.insertTriple(TripleV2(e2, result['pp']['value'], result['ans']['value']))
        res.insertTriple(TripleV2(e3, result['ppp']['value'], result['int']['value']))
        res.insertTriple(TripleV2(result['int']['value'], result['pppp']['value'], result['ans']['value']))
        ans.append(res)
    return ans

def helper6(e1,e2,e3):
    ans = []
    '''
    ?e1 - p1 - ?ans
    ?e2 - p2 - ?ans
    ?e3 - p3 - ?int
    ?ans - p4 - ?int
    '''
    queryString = "SELECT distinct ?ans ?p ?pp ?ppp ?pppp ?int WHERE {<" + e1 +"> ?p ?ans. <" + e2 + "> ?pp ?ans. <" + e3 + "> ?ppp ?int. ?ans ?pppp ?int filter(!regex(str(?p), 'wikiPage') && !regex(str(?pp), 'wikiPage') && !regex(str(?ppp), 'wikiPage') && !regex(str(?pppp), 'wikiPage') && !regex(str(?p), '#type') && !regex(str(?pp), '#type') && !regex(str(?ppp), '#type') && !regex(str(?pppp), '#type'))}"
    #print(queryString)
    sparql.setQuery(queryString)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    #print(queryString)
    for result in results["results"]["bindings"]:
        res = resultGraph()
        
        ansTp = TripleV2(e1, result['p']['value'], result['ans']['value'])
        res.setAnsTriple(ansTp)
        res.insertTriple(ansTp)
        
        res.insertTriple(TripleV2(e2, result['pp']['value'], result['ans']['value']))
        res.insertTriple(TripleV2(e3, result['ppp']['value'], result['int']['value']))
        res.insertTriple(TripleV2(result['ans']['value'], result['pppp']['value'], result['int']['value']))
        ans.append(res)
    return ans

def helper7(e1,e2,e3):
    ans = []
    '''
    ?ans - p1 - e1
    ?ans - p2 - e2
    ?int - p3 - e3
    ?ans - p4 - ?int
    '''
    queryString = "SELECT distinct ?ans ?p ?pp ?ppp ?pppp ?int WHERE {?ans ?p <" + e1 +">. ?ans ?pp <" + e2 + ">. ?int ?ppp <" + e3 + ">. ?ans ?pppp ?int filter(!regex(str(?p), 'wikiPage') && !regex(str(?pp), 'wikiPage') && !regex(str(?ppp), 'wikiPage') && !regex(str(?pppp), 'wikiPage') && !regex(str(?p), '#type') && !regex(str(?pp), '#type') && !regex(str(?ppp), '#type') && !regex(str(?pppp), '#type'))}"
    #print(queryString)
    sparql.setQuery(queryString)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    #print(queryString)
    for result in results["results"]["bindings"]:
        res = resultGraph()
        
        ansTp = TripleV2(e1, result['p']['value'], result['ans']['value'])
        res.setAnsTriple(ansTp)
        
        res.insertTriple(TripleV2(result['ans']['value'], result['p']['value'], e1))
        res.insertTriple(TripleV2(result['ans']['value'], result['pp']['value'], e2))
        res.insertTriple(TripleV2(result['int']['value'], result['ppp']['value'], e3))
        res.insertTriple(TripleV2(result['ans']['value'], result['pppp']['value'], result['int']['value']))
        ans.append(res)
    return ans

def helper8(e1,e2,e3):
    ans = []
    '''
    ?ans - p1 - e1
    ?ans - p2 - e2
    ?int - p3 - e3
    ?int - p4 - ?ans
    '''
    queryString = "SELECT distinct ?ans ?p ?pp ?ppp ?pppp ?int WHERE {?ans ?p <" + e1 +">. ?ans ?pp <" + e2 + ">. ?int ?ppp <" + e3 + ">. ?int ?pppp ?ans filter(!regex(str(?p), 'wikiPage') && !regex(str(?pp), 'wikiPage') && !regex(str(?ppp), 'wikiPage') && !regex(str(?pppp), 'wikiPage') && !regex(str(?p), '#type') && !regex(str(?pp), '#type') && !regex(str(?ppp), '#type') && !regex(str(?pppp), '#type'))}"
    #print(queryString)
    sparql.setQuery(queryString)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    #print(queryString)
    for result in results["results"]["bindings"]:
        res = resultGraph()
        res.insertTriple(TripleV2(result['ans']['value'], result['p']['value'], e1))
        res.insertTriple(TripleV2(result['ans']['value'], result['pp']['value'], e2))
        res.insertTriple(TripleV2(result['int']['value'], result['ppp']['value'], e3))
        
        ansTp = TripleV2(result['int']['value'], result['pppp']['value'], result['ans']['value'])
        res.setAnsTriple(ansTp)
        res.insertTriple(ansTp)
        
        ans.append(res)
    return ans

def helper9(e1,e2,e3):
    ans = []
    '''
    ?ans - p1 - e1
    ?e2 - p2 - ?ans
    ?int - p3 - e3
    ?int - p4 - ?ans
    '''
    queryString = "SELECT distinct ?ans ?p ?pp ?ppp ?pppp ?int WHERE {?ans ?p <" + e1 +">. <" + e2 + "> ?pp ?ans. ?int ?ppp <" + e3 + ">. ?int ?pppp ?ans filter(!regex(str(?p), 'wikiPage') && !regex(str(?pp), 'wikiPage') && !regex(str(?ppp), 'wikiPage') && !regex(str(?pppp), 'wikiPage') && !regex(str(?p), '#type') && !regex(str(?pp), '#type') && !regex(str(?ppp), '#type') && !regex(str(?pppp), '#type'))}"
    #print(queryString)
    sparql.setQuery(queryString)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    #print(queryString)
    for result in results["results"]["bindings"]:
        res = resultGraph()
        res.insertTriple(TripleV2(result['ans']['value'], result['p']['value'], e1))
        res.insertTriple(TripleV2(e2, result['pp']['value'], result['ans']['value']))
        res.insertTriple(TripleV2(result['int']['value'], result['ppp']['value'], e3))
        
        ansTp = TripleV2(result['int']['value'], result['pppp']['value'], result['ans']['value'])
        res.setAnsTriple(ansTp)
        res.insertTriple(ansTp)
        
        ans.append(res)
    return ans

def helper10(e1,e2,e3):
    ans = []
    '''
    ?ans - p1 - e1
    ?e2 - p2 - ?ans
    ?int - p3 - e3
    ?ans - p4 - ?int
    '''
    queryString = "SELECT distinct ?ans ?p ?pp ?ppp ?pppp ?int WHERE {?ans ?p <" + e1 +">. <" + e2 + "> ?pp ?ans. ?int ?ppp <" + e3 + ">. ?ans ?pppp ?int filter(!regex(str(?p), 'wikiPage') && !regex(str(?pp), 'wikiPage') && !regex(str(?ppp), 'wikiPage') && !regex(str(?pppp), 'wikiPage') && !regex(str(?p), '#type') && !regex(str(?pp), '#type') && !regex(str(?ppp), '#type') && !regex(str(?pppp), '#type'))}"
    #print(queryString)
    sparql.setQuery(queryString)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    #print(queryString)
    for result in results["results"]["bindings"]:
        res = resultGraph()
        res.insertTriple(TripleV2(result['ans']['value'], result['p']['value'], e1))
        
        ansTp = TripleV2(e2, result['pp']['value'], result['ans']['value'])
        res.setAnsTriple(ansTp)
        res.insertTriple(ansTp)
        
        res.insertTriple(TripleV2(result['int']['value'], result['ppp']['value'], e3))
        res.insertTriple(TripleV2(result['ans']['value'], result['pppp']['value'], result['int']['value']))
        ans.append(res)
    return ans

def helper11(e1,e2,e3):
    ans = []
    '''
    ?ans - p1 - e1
    ?ans - p2 - e2
    ?e3 - p3 - ?int
    ?ans - p4 - ?int
    '''
    queryString = "SELECT distinct ?ans ?p ?pp ?ppp ?pppp ?int WHERE {?ans ?p <" + e1 +">. ?ans ?pp <" + e2 + ">. <" + e3 + "> ?ppp ?int. ?ans ?pppp ?int filter(!regex(str(?p), 'wikiPage') && !regex(str(?pp), 'wikiPage') && !regex(str(?ppp), 'wikiPage') && !regex(str(?pppp), 'wikiPage') && !regex(str(?p), '#type') && !regex(str(?pp), '#type') && !regex(str(?ppp), '#type') && !regex(str(?pppp), '#type'))}"
    #print(queryString)
    sparql.setQuery(queryString)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    #print(queryString)
    for result in results["results"]["bindings"]:
        res = resultGraph()
        res.insertTriple(TripleV2(result['ans']['value'], result['p']['value'], e1))
        res.insertTriple(TripleV2(result['ans']['value'], result['pp']['value'], e2))
        res.insertTriple(TripleV2(e3, result['ppp']['value'], result['int']['value']))
        res.insertTriple(TripleV2(result['ans']['value'], result['pppp']['value'], result['int']['value']))
        
        ansTp = TripleV2(e1,result['p']['value'], result['ans']['value'])
        res.setAnsTriple(ansTp)
        
        ans.append(res)
    return ans

def helper12(e1,e2,e3):
    ans = []
    '''
    ?ans - p1 - e1
    ?ans - p2 - e2
    ?e3 - p3 - ?int
    ?int - p4 - ?ans
    '''
    queryString = "SELECT distinct ?ans ?p ?pp ?ppp ?pppp ?int WHERE {?ans ?p <" + e1 +">. ?ans ?pp <" + e2 + ">. <" + e3 + "> ?ppp ?int. ?int ?pppp ?ans filter(!regex(str(?p), 'wikiPage') && !regex(str(?pp), 'wikiPage') && !regex(str(?ppp), 'wikiPage') && !regex(str(?pppp), 'wikiPage') && !regex(str(?p), '#type') && !regex(str(?pp), '#type') && !regex(str(?ppp), '#type') && !regex(str(?pppp), '#type'))}"
    #print(queryString)
    sparql.setQuery(queryString)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    #print(queryString)
    for result in results["results"]["bindings"]:
        res = resultGraph()
        res.insertTriple(TripleV2(result['ans']['value'], result['p']['value'], e1))
        res.insertTriple(TripleV2(result['ans']['value'], result['pp']['value'], e2))
        res.insertTriple(TripleV2(e3, result['ppp']['value'], result['int']['value']))
        
        ansTp = TripleV2(result['int']['value'], result['pppp']['value'], result['ans']['value'])
        res.insertTriple(ansTp)
        res.setAnsTriple(ansTp)
        
        ans.append(res)
    return ans

def helper13(e1,e2,e3):
    ans = []
    '''
    ?ans - p1 - e1
    ?e2 - p2 - ?ans
    ?e3 - p3 - ?int
    ?int - p4 - ?ans
    '''
    queryString = "SELECT distinct ?ans ?p ?pp ?ppp ?pppp ?int WHERE {?ans ?p <" + e1 +">. <" + e2 + "> ?pp ?ans. <" + e3 + "> ?ppp ?int. ?int ?pppp ?ans filter(!regex(str(?p), 'wikiPage') && !regex(str(?pp), 'wikiPage') && !regex(str(?ppp), 'wikiPage') && !regex(str(?pppp), 'wikiPage') && !regex(str(?p), '#type') && !regex(str(?pp), '#type') && !regex(str(?ppp), '#type') && !regex(str(?pppp), '#type'))}"
    #print(queryString)
    sparql.setQuery(queryString)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    #print(queryString)
    for result in results["results"]["bindings"]:
        res = resultGraph()
        res.insertTriple(TripleV2(result['ans']['value'], result['p']['value'], e1))
        
        ansTp = TripleV2(e2, result['pp']['value'], result['ans']['value'])
        res.insertTriple(ansTp)
        res.setAnsTriple(ansTp)
        
        res.insertTriple(TripleV2(e3, result['ppp']['value'], result['int']['value']))
        res.insertTriple(TripleV2(result['int']['value'], result['pppp']['value'], result['ans']['value']))
        ans.append(res)
    return ans

def helper14(e1,e2,e3):
    ans = []
    '''
    ?ans - p1 - e1
    ?e2 - p2 - ?ans
    ?e3 - p3 - ?int
    ?ans - p4 - ?int
    '''
    queryString = "SELECT distinct ?ans ?p ?pp ?ppp ?pppp ?int WHERE {?ans ?p <" + e1 +">. <" + e2 + "> ?pp ?ans. <" + e3 + "> ?ppp ?int. ?ans ?pppp ?int filter(!regex(str(?p), 'wikiPage') && !regex(str(?pp), 'wikiPage') && !regex(str(?ppp), 'wikiPage') && !regex(str(?pppp), 'wikiPage') && !regex(str(?p), '#type') && !regex(str(?pp), '#type') && !regex(str(?ppp), '#type') && !regex(str(?pppp), '#type'))}"
    #print(queryString)
    sparql.setQuery(queryString)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    #print(queryString)
    for result in results["results"]["bindings"]:
        res = resultGraph()
        res.insertTriple(TripleV2(result['ans']['value'], result['p']['value'], e1))
        
        ansTp = TripleV2(e2, result['pp']['value'], result['ans']['value'])
        res.insertTriple(ansTp)
        res.setAnsTriple(ansTp)
        
        res.insertTriple(TripleV2(e3, result['ppp']['value'], result['int']['value']))
        res.insertTriple(TripleV2(result['ans']['value'], result['pppp']['value'], result['int']['value']))
        ans.append(res)
    return ans

testRes = threeEntityTemp()
for x in testRes:
    print(x.getAnsTriple())

http://dbpedia.org/resource/Viking_Press  ---  http://dbpedia.org/ontology/publisher  ---  http://dbpedia.org/resource/On_the_Road(1 : None)
http://dbpedia.org/resource/Viking_Press  ---  http://dbpedia.org/ontology/publisher  ---  http://dbpedia.org/resource/Atop_an_Underwood:_Early_Stories_and_Other_Writings(1 : None)
http://dbpedia.org/resource/Viking_Press  ---  http://dbpedia.org/ontology/publisher  ---  http://dbpedia.org/resource/Door_Wide_Open(1 : None)
http://dbpedia.org/resource/Viking_Press  ---  http://dbpedia.org/ontology/publisher  ---  http://dbpedia.org/resource/Atop_an_Underwood:_Early_Stories_and_Other_Writings(1 : None)
http://dbpedia.org/resource/Viking_Press  ---  http://dbpedia.org/ontology/publisher  ---  http://dbpedia.org/resource/On_the_Road(1 : None)
http://dbpedia.org/resource/Viking_Press  ---  http://dbpedia.org/ontology/publisher  ---  http://dbpedia.org/resource/On_the_Road(1 : None)
http://dbpedia.org/resource/Viking_Press  ---  http://dbpedia.org/ontol

http://dbpedia.org/resource/The_Western_Lands  ---  http://dbpedia.org/ontology/author  ---  http://dbpedia.org/resource/William_S._Burroughs(0.47019529365202506 : written)
http://dbpedia.org/resource/Sailor_Song  ---  http://dbpedia.org/ontology/author  ---  http://dbpedia.org/resource/Ken_Kesey(0.47019529365202506 : written)
http://dbpedia.org/resource/Sometimes_a_Great_Notion  ---  http://dbpedia.org/ontology/author  ---  http://dbpedia.org/resource/Ken_Kesey(0.47019529365202506 : written)
http://dbpedia.org/resource/Cities_of_the_Red_Night  ---  http://dbpedia.org/ontology/author  ---  http://dbpedia.org/resource/William_S._Burroughs(0.47019529365202506 : written)
http://dbpedia.org/resource/Queer_(novel)  ---  http://dbpedia.org/ontology/author  ---  http://dbpedia.org/resource/William_S._Burroughs(0.47019529365202506 : written)
http://dbpedia.org/resource/My_Education:_A_Book_of_Dreams  ---  http://dbpedia.org/ontology/author  ---  http://dbpedia.org/resource/William_S._Burroughs

In [31]:
def testTemp(i):
    
    labels = lbs[i]
    
    resultTriple = []
    
    e1 = labels[0]
    e2 = labels[1]
    
    keywords = keywordsList[i]
    
    '''
    ?ans - p - e1. ?ans - pp - e2
    '''
    queryString = "SELECT distinct ?ans ?p ?pp WHERE {?ans ?p <" + e1 +">. ?ans ?pp <" + e2 + ">. filter(!regex(str(?p), 'wikiPage') && !regex(str(?pp), 'wikiPage'))}"
    sparql.setQuery(queryString)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    #print(queryString)
    for result in results["results"]["bindings"]:
        res = resultGraph()
        ansTp = TripleV2(e1, result['p']['value'], result['ans']['value'])
        
        res.setAnsTriple(ansTp)
        res.insertTriple(TripleV2(result['ans']['value'], result['p']['value'], e1))
        res.insertTriple(TripleV2(result['ans']['value'], result['pp']['value'], e2))
        
        resultTriple.append(res)
        #resultTriple.append(TRIPLE(e1, result['p']['value'], result['ans']['value'], e1))
        #print(result['ans']['value'] + ' ---- ' + result['p']['value']  + ' ---- ' + result['pp']['value'])
        
        
    '''
    e1 - p - ?ans. e2 - pp - ?ans
    '''
    queryString = "SELECT distinct ?ans ?p ?pp WHERE {<" + e1 +"> ?p ?ans. <" + e2 + "> ?pp ?ans. filter(!regex(str(?p), 'wikiPage') && !regex(str(?pp), 'wikiPage') && !regex(str(?p), '#type') && !regex(str(?pp), '#type'))}"
    sparql.setQuery(queryString)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    #print(queryString)
    for result in results["results"]["bindings"]:
        
        res = resultGraph()
        ansTp = TripleV2(e1, result['p']['value'], result['ans']['value'])
        
        res.setAnsTriple(ansTp)
        res.insertTriple(ansTp)
        res.insertTriple(TripleV2(e2, result['pp']['value'], result['ans']['value']))
        resultTriple.append(res)
        #print(result['ans']['value'] + ' ---- ' + result['p']['value']  + ' ---- ' + result['pp']['value'])
        #resultTriple.append(TRIPLE(e1, result['p']['value'], result['ans']['value'], e1))
    
    
    '''
    e1 - p - ?ans. ?ans - pp - e2
    '''
    queryString = "SELECT distinct ?ans ?p ?pp WHERE {<" + e1 +"> ?p ?ans. ?ans ?pp <" + e2 + ">. filter(!regex(str(?p), 'wikiPage') && !regex(str(?pp), 'wikiPage') && !regex(str(?p), '#type') && !regex(str(?pp), '#type'))}"
    sparql.setQuery(queryString)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    #print(queryString)
    for result in results["results"]["bindings"]:
        res = resultGraph()
        ansTp = TripleV2(e1, result['p']['value'], result['ans']['value'])
        res.setAnsTriple(ansTp)
        res.insertTriple(ansTp)
        res.insertTriple(TripleV2(result['ans']['value'], result['pp']['value'], e2))
        resultTriple.append(res)
        
        #print(result['ans']['value'] + ' ---- ' + result['p']['value']  + ' ---- ' + result['pp']['value'])
        #resultTriple.append(TRIPLE(e1, result['p']['value'], result['ans']['value'], e1))
    
    
    '''
    ?ans - p - e1. e2 - pp - ?ans
    '''
    queryString = "SELECT distinct ?ans ?p ?pp WHERE {?ans ?p <" + e1 +">. <" + e2 + "> ?pp ?ans. filter(!regex(str(?p), 'wikiPage') && !regex(str(?pp), 'wikiPage') && !regex(str(?p), '#type') && !regex(str(?pp), '#type'))}"
    sparql.setQuery(queryString)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    #print(queryString)
    for result in results["results"]["bindings"]:
        res = resultGraph()
        
        ansTp = TripleV2(e2, result['pp']['value'], result['ans']['value'])
        res.setAnsTriple(ansTp)
        res.insertTriple(TripleV2(result['ans']['value'], result['p']['value'], e1))
        res.insertTriple(ansTp)
        
        resultTriple.append(res)
        #print(result['ans']['value'] + ' ---- ' + result['p']['value']  + ' ---- ' + result['pp']['value'])
        #resultTriple.append(TRIPLE(e1, result['p']['value'], result['ans']['value'], e1))
    
    for x in resultTriple:
        x.computeCost(keywords, calculateCost2)
    resultTriple.sort()
    
    return resultTriple
for x in testTemp(16):
    print(x)
    print('--------')

http://dbpedia.org/resource/On_the_Road  ---  http://dbpedia.org/ontology/publisher  ---  http://dbpedia.org/resource/Viking_Press(1.0000014804076698 : publisher)
http://dbpedia.org/resource/On_the_Road  ---  http://dbpedia.org/ontology/author  ---  http://dbpedia.org/resource/Jack_Kerouac(1.00000148040767 : author)

--------
http://dbpedia.org/resource/Atop_an_Underwood:_Early_Stories_and_Other_Writings  ---  http://dbpedia.org/ontology/publisher  ---  http://dbpedia.org/resource/Viking_Press(1.0000014804076698 : publisher)
http://dbpedia.org/resource/Atop_an_Underwood:_Early_Stories_and_Other_Writings  ---  http://dbpedia.org/ontology/author  ---  http://dbpedia.org/resource/Jack_Kerouac(1.00000148040767 : author)

--------
http://dbpedia.org/resource/Door_Wide_Open  ---  http://dbpedia.org/ontology/publisher  ---  http://dbpedia.org/resource/Viking_Press(1.0000014804076698 : publisher)
http://dbpedia.org/resource/Door_Wide_Open  ---  http://dbpedia.org/ontology/author  ---  http://d

In [66]:
def testTemp(i):
    
    labels = lbs[i]
    
    resultTriple = []
    
    e1 = labels[0]
    e2 = labels[1]
    
    
    '''
    ?ans - p - e1. ?ans - pp - e2
    '''
    queryString = "SELECT distinct ?ans ?p ?pp WHERE {?ans ?p <" + e1 +">. ?ans ?pp <" + e2 + ">. filter(!regex(str(?p), 'wikiPage') && !regex(str(?pp), 'wikiPage'))}"
    sparql.setQuery(queryString)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    #print(queryString)
    for result in results["results"]["bindings"]:
        resultTriple.append(TRIPLE(e1, result['p']['value'], result['ans']['value'], e1))
        #print(result['ans']['value'] + ' ---- ' + result['p']['value']  + ' ---- ' + result['pp']['value'])
        
        
    '''
    e1 - p - ?ans. e2 - pp - ?ans
    '''
    queryString = "SELECT distinct ?ans ?p ?pp WHERE {<" + e1 +"> ?p ?ans. <" + e2 + "> ?pp ?ans. filter(!regex(str(?p), 'wikiPage') && !regex(str(?pp), 'wikiPage') && !regex(str(?p), '#type') && !regex(str(?pp), '#type'))}"
    sparql.setQuery(queryString)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    #print(queryString)
    for result in results["results"]["bindings"]:
        #print(result['ans']['value'] + ' ---- ' + result['p']['value']  + ' ---- ' + result['pp']['value'])
        resultTriple.append(TRIPLE(e1, result['p']['value'], result['ans']['value'], e1))
    
    
    '''
    e1 - p - ?ans. ?ans - pp - e2
    '''
    queryString = "SELECT distinct ?ans ?p ?pp WHERE {<" + e1 +"> ?p ?ans. ?ans ?pp <" + e2 + ">. filter(!regex(str(?p), 'wikiPage') && !regex(str(?pp), 'wikiPage') && !regex(str(?p), '#type') && !regex(str(?pp), '#type'))}"
    sparql.setQuery(queryString)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    #print(queryString)
    for result in results["results"]["bindings"]:
        #print(result['ans']['value'] + ' ---- ' + result['p']['value']  + ' ---- ' + result['pp']['value'])
        resultTriple.append(TRIPLE(e1, result['p']['value'], result['ans']['value'], e1))
    
    
    '''
    ?ans - p - e1. e2 - pp - ?ans
    '''
    queryString = "SELECT distinct ?ans ?p ?pp WHERE {?ans ?p <" + e1 +">. <" + e2 + "> ?pp ?ans. filter(!regex(str(?p), 'wikiPage') && !regex(str(?pp), 'wikiPage') && !regex(str(?p), '#type') && !regex(str(?pp), '#type'))}"
    sparql.setQuery(queryString)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    #print(queryString)
    for result in results["results"]["bindings"]:
        #print(result['ans']['value'] + ' ---- ' + result['p']['value']  + ' ---- ' + result['pp']['value'])
        resultTriple.append(TRIPLE(e1, result['p']['value'], result['ans']['value'], e1))
    
    
    return resultTriple
for x in testTemp(77):
    print(x)

http://dbpedia.org/resource/Electronics  ---  http://dbpedia.org/ontology/industry  ---  http://dbpedia.org/resource/Aviation_Industry_Corporation_of_China(0)
http://dbpedia.org/resource/Electronics  ---  http://dbpedia.org/ontology/product  ---  http://dbpedia.org/resource/Aviation_Industry_Corporation_of_China(0)
http://dbpedia.org/resource/Electronics  ---  http://dbpedia.org/ontology/industry  ---  http://dbpedia.org/resource/Lenovo(0)
http://dbpedia.org/resource/Electronics  ---  http://dbpedia.org/ontology/product  ---  http://dbpedia.org/resource/China_Aerospace_Science_and_Technology_Corporation(0)


In [19]:
i = 142
print(getAnswerType(keywordsList[i],pdic))
print(keywordsList[i])

None
['Country', 'language']


In [55]:
import timeit

with open('fwres.txt','w',encoding='utf-8') as fw:
    for x in range(16,17):
        fw.write('------------\n')
        fw.write(str(x) + '\n')
        fw.write('------------\n')
        expandedGraph = set()
        answerPool = set()
        labels = lbs[x]
        keywords = keywordsList[x]

        if len(keywords) == 1 and keywords[0]=='':
            print('keywords length 0')
            continue
        
        for label in labels:
            sparql.setQuery(createQueryString(label))
            sparql.setReturnFormat(JSON)
            results = sparql.query().convert()
            #print(queryString)
            for result in results["results"]["bindings"]:
                if 's' in result:
                    newtp = TRIPLE(label, result['p']['value'], result['s']['value'], label)
                    newtp.insertCost(keywords)
                    expandedGraph.add(newtp)

                    #add duplicatedItem
                    possibleDup = checkDup(expandedGraph,newtp)
                    if len(possibleDup) > 0:
                        for x in possibleDup:
                            answerPool.add(x)
                        answerPool.add(newtp)

                else:
                    newtp = TRIPLE(label, result['j']['value'], result['k']['value'], label)
                    newtp.insertCost(keywords)
                    expandedGraph.add(newtp)

                    #add duplicatedItem
                    possibleDup = checkDup(expandedGraph,newtp)
                    if len(possibleDup) > 0:
                        for x in possibleDup:
                            answerPool.add(x)
                        answerPool.add(newtp)
                        
        
        #####################
        
        #if steiner tree after 1 hop, print answer
        eg3 = list(expandedGraph)
        eg3.sort()
        topK = 5
        ap3 = list(answerPool)
        ap3.sort()
        
        
        #for x in fetchTopK(eg2,topK):
        #        fw.write(str(x) + '\n')
        
        if len(ap3) > 0:
            for x in fetchTopK(ap3,topK):
                fw.write(str(x) + '\n')
            continue
        #####################
        
        
        
        #####################
        # set a threshold and expand for one more hop for those entities
        twoHops = []
        thThreshold = 0.5
        for x in expandedGraph:
            if x.getCost()> thThreshold and x.getObj().startswith('http://dbpedia.org/resource/'):
                twoHops.append(x)
        #print(twoHops)
        
        #sort twoHops
        twoHops.sort()
        
        for x in twoHops:
            print(x)
        
        timeTotal = 0
        
        
        
        #####################
        tpc = 0
        while tpc < len(twoHops) and timeTotal < 120:
            #get StartTime
            tstart = timeit.default_timer()
            
            label = twoHops[tpc].getObj()
            sparql.setQuery(createQueryString(label))
            sparql.setReturnFormat(JSON)
            results = sparql.query().convert()
            #print(queryString)
            for result in results["results"]["bindings"]:
                if 's' in result:
                    newtp = TRIPLE(label, result['p']['value'], result['s']['value'], label)
                    newtp.setPreviousTriple(twoHops[tpc])
                    newtp.insertCost(keywords)
                    expandedGraph.add(newtp)

                    #add duplicatedItem
                    possibleDup = checkDup(expandedGraph,newtp)
                    if len(possibleDup) > 0:
                        for x in possibleDup:
                            answerPool.add(x)
                        answerPool.add(newtp)

                else:
                    newtp = TRIPLE(label, result['j']['value'], result['k']['value'], label)
                    newtp.setPreviousTriple(twoHops[tpc])
                    newtp.insertCost(keywords)
                    expandedGraph.add(newtp)

                    #add duplicatedItem
                    possibleDup = checkDup(expandedGraph,newtp)
                    if len(possibleDup) > 0:
                        for x in possibleDup:
                            answerPool.add(x)
                        answerPool.add(newtp)
            tpc += 1
            
            #get endTime
            tend = timeit.default_timer()
            timeTotal += tend - tstart
            #rint('Time: ', tend - tstart)
        #####################
        
        
        
        eg2 = list(expandedGraph)
        eg2.sort()
        topK = 5
        ap2 = list(answerPool)
        ap2.sort()
        
        
        #for x in fetchTopK(eg2,topK):
        #        fw.write(str(x) + '\n')
        '''
        if len(ap2) > 0:
            for x in fetchTopK(ap2,topK):
                fw.write(str(x) + '\n')
        else:
            for x in fetchTopK(eg2,topK):
                fw.write(str(x) + '\n')
        '''
        for x in fetchTopK(eg2,topK):
                fw.write(str(x) + '\n')

In [62]:
def removeZero(li):
    newLi = []
    for x in li:
        if x.getProb() != 0:
            newLi.append(x)
    return newLi

In [75]:
import timeit

with open('fwres.txt','w',encoding='utf-8') as fw:
    for x in range(0,215):
        fw.write('------------\n')
        fw.write(str(x) + '\n')
        fw.write('------------\n')
        expandedGraph = set()
        answerPool = set()
        labels = lbs[x]
        keywords = keywordsList[x]

        if len(keywords) == 1 and keywords[0]=='':
            print('keywords length 0')
            continue
        
        
        """
        if num of labels == 2, use templates
        """
        
        if len(labels) == 2:
            print(str(x) + ' has num of labels 2')
            res = testTemp(x)
            if len(res) != 0:
                for k in res:
                    fw.write(str(k) + '\n')
                continue
        
        
        for label in labels:
            sparql.setQuery(createQueryString(label))
            sparql.setReturnFormat(JSON)
            results = sparql.query().convert()
            #print(queryString)
            for result in results["results"]["bindings"]:
                if 's' in result:
                    newtp = TRIPLE(label, result['p']['value'], result['s']['value'], label)
                    newtp.insertCost(keywords)
                    expandedGraph.add(newtp)

                    #add duplicatedItem
                    possibleDup = checkDup(expandedGraph,newtp)
                    if len(possibleDup) > 0:
                        for x in possibleDup:
                            answerPool.add(x)
                        answerPool.add(newtp)

                else:
                    newtp = TRIPLE(label, result['j']['value'], result['k']['value'], label)
                    newtp.insertCost(keywords)
                    expandedGraph.add(newtp)

                    #add duplicatedItem
                    possibleDup = checkDup(expandedGraph,newtp)
                    if len(possibleDup) > 0:
                        for x in possibleDup:
                            answerPool.add(x)
                        answerPool.add(newtp)
                        
        
        #################
        #get answer types
        answerType = getAnswerType(keywords,pdic)
        print(answerType)
        
        #####################
        
        #if steiner tree after 1 hop, print answer
        eg3 = list(expandedGraph)
        eg3.sort()
        topK = 5
        ap3 = list(answerPool)
        ap3.sort()
        
        
        #for x in fetchTopK(eg2,topK):
        #        fw.write(str(x) + '\n')
        typeChecked = []
        if len(ap3) > 0:
            answers = fetchTopK(ap3,topK)
            if answerType != 'None':
                for x in answers:
                    if filterTypeQuery(x.getObj(),answerType):
                        print('is type')
                        typeChecked.append(x)
                        #fw.write(str(x) + '\n')
                if len(typeChecked) == 0:
                    for x in answers:
                        fw.write(str(x) + '\n')
                else:
                    for x in typeChecked:
                        fw.write(str(x) + '\n')
            else:
                for x in answers:
                    fw.write(str(x) + '\n')
            continue
        #####################
        
        
        
        #####################
        # set a threshold and expand for one more hop for those entities
        twoHops = []
        thThreshold = 0.5
        for x in expandedGraph:
            if x.getCost()> thThreshold and x.getObj().startswith('http://dbpedia.org/resource/'):
                twoHops.append(x)
        #print(twoHops)
        
        #sort twoHops
        twoHops.sort()
        
        for x in twoHops:
            print(x)
        
        timeTotal = 0
        
        
        
        #####################
        tpc = 0
        while tpc < len(twoHops) and timeTotal < 120:
            #get StartTime
            tstart = timeit.default_timer()
            
            label = twoHops[tpc].getObj()
            sparql.setQuery(createQueryString(label))
            sparql.setReturnFormat(JSON)
            results = sparql.query().convert()
            #print(queryString)
            for result in results["results"]["bindings"]:
                if 's' in result:
                    newtp = TRIPLE(label, result['p']['value'], result['s']['value'], label)
                    newtp.setPreviousTriple(twoHops[tpc])
                    newtp.insertCost(keywords)
                    expandedGraph.add(newtp)

                    #add duplicatedItem
                    possibleDup = checkDup(expandedGraph,newtp)
                    if len(possibleDup) > 0:
                        for x in possibleDup:
                            answerPool.add(x)
                        answerPool.add(newtp)

                else:
                    newtp = TRIPLE(label, result['j']['value'], result['k']['value'], label)
                    newtp.setPreviousTriple(twoHops[tpc])
                    newtp.insertCost(keywords)
                    expandedGraph.add(newtp)

                    #add duplicatedItem
                    possibleDup = checkDup(expandedGraph,newtp)
                    if len(possibleDup) > 0:
                        for x in possibleDup:
                            answerPool.add(x)
                        answerPool.add(newtp)
            tpc += 1
            
            #get endTime
            tend = timeit.default_timer()
            timeTotal += tend - tstart
            #rint('Time: ', tend - tstart)
        #####################
        
        
        
        eg2 = list(expandedGraph)
        eg2.sort()
        topK = 5
        ap2 = list(answerPool)
        ap2.sort()
        
        
        #for x in fetchTopK(eg2,topK):
        #        fw.write(str(x) + '\n')
        '''
        if len(ap2) > 0:
            for x in fetchTopK(ap2,topK):
                fw.write(str(x) + '\n')
        else:
            for x in fetchTopK(eg2,topK):
                fw.write(str(x) + '\n')
        '''
        typeChecked = []

        answers = removeZero(fetchTopK(eg2,topK))
        
        
        
        
        if answerType != 'None':
            for x in answers:
                if filterTypeQuery(x.getObj(),answerType):
                    print(x.getObj() + ' is type: ' + answerType)
                    typeChecked.append(x)
                    #fw.write(str(x) + '\n')
            if len(typeChecked) == 0:
                for x in answers:
                    fw.write(str(x) + '\n')
            else:
                for x in typeChecked:
                    fw.write(str(x) + '\n')
        else:
            for x in answers:
                fw.write(str(x) + '\n')
                    
        #for x in fetchTopK(eg2,topK):
        #        fw.write(str(x) + '\n')

None
None
http://dbpedia.org/resource/Elton_John  ---  http://dbpedia.org/ontology/musicBy  ---  http://dbpedia.org/resource/Billy_Elliot_the_Musical(1.0)
http://dbpedia.org/resource/Elton_John  ---  http://dbpedia.org/ontology/musicBy  ---  http://dbpedia.org/resource/Aida_(musical)(1.0)
http://dbpedia.org/resource/Elton_John  ---  http://dbpedia.org/ontology/musicBy  ---  http://dbpedia.org/resource/The_Lion_King_(musical)(1.0)
http://dbpedia.org/resource/Elton_John  ---  http://dbpedia.org/ontology/musicBy  ---  http://dbpedia.org/resource/Lestat_(musical)(1.0)
http://dbpedia.org/resource/Elton_John  ---  http://dbpedia.org/ontology/musicBy  ---  http://dbpedia.org/resource/The_Lion_King_Jr_(musical)(1.0)
http://dbpedia.org/resource/Elton_John  ---  http://dbpedia.org/property/music  ---  http://dbpedia.org/resource/Grammy_Nominees(0.71)
http://dbpedia.org/resource/Elton_John  ---  http://dbpedia.org/property/music  ---  http://dbpedia.org/resource/Versão_Acústica_4(0.71)
http://dbp

http://dbpedia.org/resource/Kassel is type: City
http://dbpedia.org/resource/Bremerhaven is type: City
None
http://dbpedia.org/resource/FC_Porto  ---  http://dbpedia.org/ontology/ground  ---  http://dbpedia.org/resource/Estádio_do_Dragão(1.0)
None
http://dbpedia.org/resource/Robert_F._Kennedy  ---  http://dbpedia.org/ontology/spouse  ---  http://dbpedia.org/resource/Ethel_Kennedy(1.0)
http://dbpedia.org/resource/Robert_F._Kennedy  ---  http://dbpedia.org/ontology/child  ---  http://dbpedia.org/resource/Kerry_Kennedy(1.0)
http://dbpedia.org/resource/Robert_F._Kennedy  ---  http://dbpedia.org/ontology/child  ---  http://dbpedia.org/resource/Joseph_P._Kennedy_Sr.(1.0)
http://dbpedia.org/resource/Robert_F._Kennedy  ---  http://dbpedia.org/ontology/child  ---  http://dbpedia.org/resource/Rose_Kennedy(1.0)
http://dbpedia.org/resource/Robert_F._Kennedy  ---  http://dbpedia.org/ontology/child  ---  http://dbpedia.org/resource/Rory_Kennedy(1.0)
http://dbpedia.org/resource/Robert_F._Kennedy  ---

http://dbpedia.org/resource/Alec_Guinness  ---  http://dbpedia.org/ontology/starring  ---  http://dbpedia.org/resource/A_Run_for_Your_Money(0.99)
http://dbpedia.org/resource/Alec_Guinness  ---  http://dbpedia.org/ontology/starring  ---  http://dbpedia.org/resource/The_Scapegoat_(1959_film)(0.99)
http://dbpedia.org/resource/Alec_Guinness  ---  http://dbpedia.org/ontology/starring  ---  http://dbpedia.org/resource/Kind_Hearts_and_Coronets(0.99)
http://dbpedia.org/resource/Alec_Guinness  ---  http://dbpedia.org/ontology/starring  ---  http://dbpedia.org/resource/Hotel_Paradiso_(film)(0.99)
http://dbpedia.org/resource/Alec_Guinness  ---  http://dbpedia.org/ontology/starring  ---  http://dbpedia.org/resource/Oliver_Twist_(1948_film)(0.99)
http://dbpedia.org/resource/Alec_Guinness  ---  http://dbpedia.org/ontology/starring  ---  http://dbpedia.org/resource/Situation_Hopeless..._But_Not_Serious(0.99)
http://dbpedia.org/resource/Alec_Guinness  ---  http://dbpedia.org/ontology/starring  ---  ht

http://dbpedia.org/resource/William_Shatner  ---  http://dbpedia.org/ontology/starring  ---  http://dbpedia.org/resource/The_Explosive_Generation(0.99)
http://dbpedia.org/resource/William_Shatner  ---  http://dbpedia.org/ontology/starring  ---  http://dbpedia.org/resource/TekWar_(TV_series)(0.99)
http://dbpedia.org/resource/William_Shatner  ---  http://dbpedia.org/ontology/starring  ---  http://dbpedia.org/resource/Airplane_II:_The_Sequel(0.99)
http://dbpedia.org/resource/William_Shatner  ---  http://dbpedia.org/ontology/starring  ---  http://dbpedia.org/resource/Shoot_or_Be_Shot(0.99)
http://dbpedia.org/resource/William_Shatner  ---  http://dbpedia.org/ontology/starring  ---  http://dbpedia.org/resource/$h*!_My_Dad_Says(0.99)
http://dbpedia.org/resource/William_Shatner  ---  http://dbpedia.org/ontology/starring  ---  http://dbpedia.org/resource/Boston_Legal(0.99)
http://dbpedia.org/resource/William_Shatner  ---  http://dbpedia.org/ontology/starring  ---  http://dbpedia.org/resource/T.

http://dbpedia.org/resource/Computer_science  ---  http://dbpedia.org/ontology/field  ---  http://dbpedia.org/resource/Chandrajit_Bajaj(1.0)
http://dbpedia.org/resource/Computer_science  ---  http://dbpedia.org/ontology/field  ---  http://dbpedia.org/resource/Mark_DeLoura(1.0)
http://dbpedia.org/resource/Computer_science  ---  http://dbpedia.org/ontology/field  ---  http://dbpedia.org/resource/Ramesh_Govindan(1.0)
http://dbpedia.org/resource/Computer_science  ---  http://dbpedia.org/ontology/field  ---  http://dbpedia.org/resource/Ian_Goodfellow(1.0)
http://dbpedia.org/resource/Computer_science  ---  http://dbpedia.org/ontology/field  ---  http://dbpedia.org/resource/Conor_McBride(1.0)
http://dbpedia.org/resource/Computer_science  ---  http://dbpedia.org/ontology/field  ---  http://dbpedia.org/resource/Giorgio_Ausiello(1.0)
http://dbpedia.org/resource/Computer_science  ---  http://dbpedia.org/ontology/field  ---  http://dbpedia.org/resource/Norm_Cox_(designer)(1.0)
http://dbpedia.org/r

...

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Book
http://dbpedia.org/resource/Muhammad_Ali  ---  http://dbpedia.org/ontology/author  ---  http://dbpedia.org/resource/The_Soul_of_a_Butterfly(1.0)
http://dbpedia.org/resource/Muhammad_Ali  ---  http://dbpedia.org/ontology/author  ---  http://dbpedia.org/resource/The_Greatest:_My_Own_Story(1.0)
http://dbpedia.org/resource/Muhammad_Ali  ---  http://dbpedia.org/ontology/writer  ---  http://dbpedia.org/resource/The_Greatest_(1977_film)(0.75)
http://dbpedia.org/resource/Muhammad_Ali  ---  http://dbpedia.org/property/title  ---  http://dbpedia.org/resource/List_of_WBC_world_champions(0.55)
http://dbpedia.org/resource/Muhammad_Ali  ---  http://dbpedia.org/property/title  ---  http://dbpedia.org/resource/List_of_undisputed_boxing_champions(0.55)
http://dbpedia.org/resource/Muhammad_Ali  ---  http://dbpedia.org/property/title  ---  http://dbpedia.org/resource/List_of_NABF_champions(0.55)
http://dbpedia.org/resource/Muhammad_Ali  ---  http://dbpedia.org/property/title  ---  http://dbpedia.org

http://dbpedia.org/resource/Walt_Disney  ---  http://dbpedia.org/ontology/creator  ---  http://dbpedia.org/resource/Julius_the_Cat(1.0)
http://dbpedia.org/resource/Walt_Disney  ---  http://dbpedia.org/ontology/creator  ---  http://dbpedia.org/resource/List_of_Jiminy_Cricket_educational_serials(1.0)
http://dbpedia.org/resource/Walt_Disney  ---  http://dbpedia.org/ontology/creator  ---  http://dbpedia.org/resource/Donald_Duck(1.0)
http://dbpedia.org/resource/Walt_Disney  ---  http://dbpedia.org/ontology/creator  ---  http://dbpedia.org/resource/Pete_Junior(1.0)
http://dbpedia.org/resource/Walt_Disney  ---  http://dbpedia.org/ontology/creator  ---  http://dbpedia.org/resource/The_Mickey_Mouse_Club(1.0)
http://dbpedia.org/resource/Walt_Disney  ---  http://dbpedia.org/ontology/creator  ---  http://dbpedia.org/resource/Mickey_Mouse(1.0)
http://dbpedia.org/resource/Walt_Disney  ---  http://dbpedia.org/ontology/creator  ---  http://dbpedia.org/resource/Snow_White_(Disney)(1.0)
http://dbpedia.o

http://dbpedia.org/resource/Oregon_wine  ---  http://dbpedia.org/ontology/growingGrape  ---  http://dbpedia.org/resource/Fiano_(grape)(1.0)
http://dbpedia.org/resource/Oregon_wine  ---  http://dbpedia.org/ontology/growingGrape  ---  http://dbpedia.org/resource/Early_Muscat(1.0)
http://dbpedia.org/resource/Oregon_wine  ---  http://dbpedia.org/ontology/growingGrape  ---  http://dbpedia.org/resource/Pinot_noir(1.0)
http://dbpedia.org/resource/Oregon_wine  ---  http://dbpedia.org/ontology/growingGrape  ---  http://dbpedia.org/resource/Marsanne(1.0)
http://dbpedia.org/resource/Oregon_wine  ---  http://dbpedia.org/ontology/growingGrape  ---  http://dbpedia.org/resource/Grenache_blanc(1.0)
http://dbpedia.org/resource/Oregon_wine  ---  http://dbpedia.org/ontology/growingGrape  ---  http://dbpedia.org/resource/Trousseau_(grape)(1.0)
http://dbpedia.org/resource/Oregon_wine  ---  http://dbpedia.org/ontology/growingGrape  ---  http://dbpedia.org/resource/Kerner_(grape)(1.0)
http://dbpedia.org/reso

http://dbpedia.org/resource/William_Goldman  ---  http://dbpedia.org/ontology/writer  ---  http://dbpedia.org/resource/Absolute_Power_(film)(0.75)
http://dbpedia.org/resource/William_Goldman  ---  http://dbpedia.org/ontology/writer  ---  http://dbpedia.org/resource/The_Ghost_and_the_Darkness(0.75)
http://dbpedia.org/resource/William_Goldman  ---  http://dbpedia.org/ontology/writer  ---  http://dbpedia.org/resource/Marathon_Man_(film)(0.75)
http://dbpedia.org/resource/William_Goldman  ---  http://dbpedia.org/ontology/writer  ---  http://dbpedia.org/resource/The_Hot_Rock_(film)(0.75)
http://dbpedia.org/resource/William_Goldman  ---  http://dbpedia.org/ontology/writer  ---  http://dbpedia.org/resource/Masquerade_(1965_film)(0.75)
http://dbpedia.org/resource/William_Goldman  ---  http://dbpedia.org/ontology/writer  ---  http://dbpedia.org/resource/Harper_(film)(0.75)
http://dbpedia.org/resource/William_Goldman  ---  http://dbpedia.org/ontology/writer  ---  http://dbpedia.org/resource/Soldi

In [57]:
for x in fetchTopK(ap3,5):
    print(x.getObj())

http://dbpedia.org/resource/Door_Wide_Open
http://dbpedia.org/resource/On_the_Road
http://dbpedia.org/resource/Atop_an_Underwood:_Early_Stories_and_Other_Writings
http://dbpedia.org/resource/Door_Wide_Open
http://dbpedia.org/resource/On_the_Road
http://dbpedia.org/resource/Atop_an_Underwood:_Early_Stories_and_Other_Writings
http://dbpedia.org/resource/On_the_Road


In [11]:
import spacy

nlp = spacy.load("en_core_web_sm")
doc = nlp("Who are the parents of the wife of Juan Carlos I?")

for token in doc:
    print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
            token.shape_, token.is_alpha, token.is_stop)

Who who PRON WP attr Xxx True True
are be AUX VBP ROOT xxx True True
the the DET DT det xxx True True
parents parent NOUN NNS attr xxxx True False
of of ADP IN prep xx True True
the the DET DT det xxx True True
wife wife NOUN NN pobj xxxx True False
of of ADP IN prep xx True True
Juan Juan PROPN NNP compound Xxxx True False
Carlos Carlos PROPN NNP compound Xxxxx True False
I -PRON- PRON PRP pobj X True True
? ? PUNCT . punct ? False False


In [ ]:
#find propn in sentences, nouns

In [22]:
with open('naturalLanguageQuestions.txt','r') as fnlq:
    nlq = fnlq.readlines()

In [118]:
nlp = spacy.load("en_core_web_sm")
count = [6,15,26,27,51,75,88,90,91,133,179,193,205]
whoq = [3,4,9,10,12,15,19,21,23,25,26,27,32,40,48,50,68,70,71,74,86,99,108,112,116,119,121,122,127,129,131,137,153,155,158,159,164]
whatq = [11,24,42,45,46,49,52,60,63,65,66,80,92,93,106,117,145,147,148,151,152,154,169,171,192,209,211]
whichq = [8,13,16,18,22,28,34,35,36,44,54,56,58,72,73]
for ct in whichq:
    doc = nlp(nlq[ct].strip())
    
    for token in doc:
        print(token.text, token.dep_, token.head.text, token.head.pos_,
            [child for child in token.children])
    '''
    for token in doc:
        if token.dep_ == 'nsubj' or token.dep_ == 'nsubjpass':
            print(token.text)
    '''
    print('------------------------')

In ROOT In ADP [does]
which det city NOUN []
city nsubj does AUX [which]
does pcomp In ADP [city, Route, end, ?]
the det Route PROPN []
Chile compound Route PROPN []
Route dobj does AUX [the, Chile]
68 nummod end NOUN []
end npadvmod does AUX [68]
? punct does AUX []
------------------------
Which det cities NOUN []
cities nsubj flow VERB [Which]
does aux flow VERB []
the det Weser PROPN []
Weser nsubj flow VERB [the]
flow ROOT flow VERB [cities, does, Weser, through, ?]
through prt flow VERB []
? punct flow VERB []
------------------------
Which det books NOUN []
books nsubjpass published VERB [Which, by]
by prep books NOUN [Kerouac]
Kerouac pobj by ADP []
were auxpass published VERB []
published ROOT published VERB [books, were, by, ?]
by agent published VERB [Press]
Viking compound Press PROPN []
Press pobj by ADP [Viking]
? punct published VERB []
------------------------
Which det museum NOUN []
museum nsubj exhibits VERB [Which]
exhibits ROOT exhibits VERB [museum, Scream, by, ?]

In [21]:
nlp = spacy.load("en_core_web_sm")
count = 8
doc = nlp(nlq[count].strip())
for token in doc:
    
    print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
            token.shape_, token.is_alpha, token.is_stop)
print('------------------------')

for chunk in doc.noun_chunks:
    print(chunk.text, chunk.root.text, chunk.root.dep_,
            chunk.root.head.text)
print('------------------------')
'''
c = 0
word = ''
while c < len(doc):
    token = doc[c]
    if token.pos_ == "PROPN":
        word += token.text + ' '
        #print(token.text)
    elif token.pos_ == "ADP" and word != '':
        word += token.text + ' '
    else:
        if word != '':
            print(word)
            word = ''
    c += 1
'''

In in ADP IN ROOT Xx True True
which which DET WDT det xxxx True True
city city NOUN NN nsubj xxxx True False
does do AUX VBZ pcomp xxxx True True
the the DET DT det xxx True True
Chile Chile PROPN NNP compound Xxxxx True False
Route Route PROPN NNP dobj Xxxxx True False
68 68 NUM CD nummod dd False False
end end NOUN NN npadvmod xxx True False
? ? PUNCT . punct ? False False
------------------------
which city city nsubj does
the Chile Route Route dobj does
------------------------


'\nc = 0\nword = \'\'\nwhile c < len(doc):\n    token = doc[c]\n    if token.pos_ == "PROPN":\n        word += token.text + \' \'\n        #print(token.text)\n    elif token.pos_ == "ADP" and word != \'\':\n        word += token.text + \' \'\n    else:\n        if word != \'\':\n            print(word)\n            word = \'\'\n    c += 1\n'

In [70]:
import spacy

nlp = spacy.load("en_core_web_sm")
#doc = nlp("Who is the daughter of Robert Kennedy married to?")
doc = nlp("Who is the mayor of the capital of French Polynesia?")
for token in doc:
    print(token.text, token.dep_, token.head.text, token.head.pos_,
            [child for child in token.children])

Who nsubj is AUX []
is ROOT is AUX [Who, mayor, ?]
the det mayor NOUN []
mayor attr is AUX [the, of]
of prep mayor NOUN [capital]
the det capital NOUN []
capital pobj of ADP [the, of]
of prep capital NOUN [Polynesia]
French compound Polynesia PROPN []
Polynesia pobj of ADP [French]
? punct is AUX []


In [121]:
with open('sparqlq.txt','r') as fq:
    qs = fq.readlines()
count = 0
li = []
while count < len(qs):
    x = qs[count]
    if ':type' in x:
        li.append(count)
    count += 1
print(li)

[1, 46, 64, 65, 67, 78, 81, 95, 103, 107, 117, 123, 124, 135, 143, 150, 151, 156, 183, 188, 194, 196, 200, 202, 212]


In [122]:
nlp = spacy.load("en_core_web_sm")
for count in li:
    doc = nlp(nlq[count].strip())
    for token in doc:
        print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
                token.shape_, token.is_alpha, token.is_stop)
    print('------------------------')

List list VERB VB ROOT Xxxx True False
all all DET PDT predet xxx True True
the the DET DT det xxx True True
musicals musical NOUN NNS dobj xxxx True False
with with ADP IN prep xxxx True True
music music NOUN NN pobj xxxx True False
by by ADP IN prep xx True True
Elton Elton PROPN NNP compound Xxxxx True False
John John PROPN NNP pobj Xxxx True False
. . PUNCT . punct . False False
------------------------
What what PRON WP attr Xxxx True True
is be AUX VBZ ROOT xx True True
the the DET DT det xxx True True
highest high ADJ JJS amod xxxx True False
mountain mountain NOUN NN nsubj xxxx True False
in in ADP IN prep xx True True
Australia Australia PROPN NNP pobj Xxxxx True False
? ? PUNCT . punct ? False False
------------------------
Did do AUX VBD aux Xxx True True
Arnold Arnold PROPN NNP compound Xxxxx True False
Schwarzenegger Schwarzenegger PROPN NNP nsubj Xxxxx True False
attend attend VERB VB ROOT xxxx True False
a a DET DT det x True True
university university NOUN NN dobj xxxx 

In [27]:
nlp = spacy.load("en_core_web_sm")

li = [7,16,27,28,52,76,89,91,92,134,180,194,206]

for count in li:
    count -= 1
    doc = nlp(nlq[count].strip())
    '''
    for token in doc:

        print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
                token.shape_, token.is_alpha, token.is_stop)
    print('------------------------')
    '''
    for chunk in doc.noun_chunks:
        print(chunk.text, chunk.root.text, chunk.root.dep_,
                chunk.root.head.text)
    for token in doc:
        if token.pos_ == "VERB":
            print(token.text)
    print('------------------------')
    '''
    vc = 0
    for token in doc:
        
        if token.pos_ == 'VERB':
            vc += 1
    if vc >= 2:
        print(count)
    '''

what city city nsubj is
the Heineken brewery brewery nsubj is
------------------------
Who Who nsubj is
the daughter daughter attr is
Robert Kennedy Kennedy pobj of
married
------------------------
Who Who nsubj is
the coach coach attr is
Ankara's ice hockey team team pobj of
------------------------
Who Who nsubj is
the mayor mayor attr is
the capital capital pobj of
French Polynesia Polynesia pobj of
------------------------
me me dative Give
all actors actors dobj Give
movies movies pobj in
William Shatner Shatner pobj by
Give
starring
directed
------------------------
How many people people nsubj live
the capital capital pobj in
Australia Australia pobj of
live
------------------------
me me dative Give
the grandchildren grandchildren dobj Give
Elvis Presley Presley pobj of
Give
------------------------
Dracula's creator creator nsubj die
die
------------------------
Which country country nsubj does
the creator creator nsubj come
Miffy Miffy pobj of
come
------------------------
Ho

In [30]:
import timeit

start = timeit.default_timer()

import spacy

nlp = spacy.load("en_core_web_sm")
#doc = nlp("Who is the daughter of Robert Kennedy married to?")
doc = nlp("Who is the mayor of the capital of French Polynesia?")
for token in doc:
    print(token.text, token.dep_, token.head.text, token.head.pos_,
            [child for child in token.children])

stop = timeit.default_timer()

print('Time: ', stop - start)
print(start)
print(stop)

Who nsubj is AUX []
is ROOT is AUX [Who, mayor, ?]
the det mayor NOUN []
mayor attr is AUX [the, of]
of prep mayor NOUN [capital]
the det capital NOUN []
capital pobj of ADP [the, of]
of prep capital NOUN [Polynesia]
French compound Polynesia PROPN []
Polynesia pobj of ADP [French]
? punct is AUX []
Time:  0.5797196999999414
1497.8622923
1498.442012


In [47]:
for x in eg3:
    print(x)

http://dbpedia.org/resource/Jack_Kerouac  ---  http://dbpedia.org/ontology/author  ---  http://dbpedia.org/resource/Doctor_Sax(1.00000148040767)
http://dbpedia.org/resource/Jack_Kerouac  ---  http://dbpedia.org/ontology/author  ---  http://dbpedia.org/resource/Pic_(novel)(1.00000148040767)
http://dbpedia.org/resource/Jack_Kerouac  ---  http://dbpedia.org/ontology/author  ---  http://dbpedia.org/resource/Satori_in_Paris(1.00000148040767)
http://dbpedia.org/resource/Jack_Kerouac  ---  http://dbpedia.org/ontology/author  ---  http://dbpedia.org/resource/Lonesome_Traveler(1.00000148040767)
http://dbpedia.org/resource/Jack_Kerouac  ---  http://dbpedia.org/ontology/author  ---  http://dbpedia.org/resource/Atop_an_Underwood:_Early_Stories_and_Other_Writings(1.00000148040767)
http://dbpedia.org/resource/Jack_Kerouac  ---  http://dbpedia.org/ontology/author  ---  http://dbpedia.org/resource/Good_Blonde_&_Others(1.00000148040767)
http://dbpedia.org/resource/Jack_Kerouac  ---  http://dbpedia.org/

http://dbpedia.org/resource/Viking_Press  ---  http://dbpedia.org/ontology/publisher  ---  http://dbpedia.org/resource/A_World_to_Win_(Sinclair_novel)(1.0000014804076698)
http://dbpedia.org/resource/Viking_Press  ---  http://dbpedia.org/ontology/publisher  ---  http://dbpedia.org/resource/Back_Roads_(novel)(1.0000014804076698)
http://dbpedia.org/resource/Viking_Press  ---  http://dbpedia.org/ontology/publisher  ---  http://dbpedia.org/resource/The_Last_Unicorn(1.0000014804076698)
http://dbpedia.org/resource/Viking_Press  ---  http://dbpedia.org/ontology/publisher  ---  http://dbpedia.org/resource/Death_of_a_Dude(1.0000014804076698)
http://dbpedia.org/resource/Viking_Press  ---  http://dbpedia.org/ontology/publisher  ---  http://dbpedia.org/resource/The_Golden_Spiders(1.0000014804076698)
http://dbpedia.org/resource/Viking_Press  ---  http://dbpedia.org/ontology/publisher  ---  http://dbpedia.org/resource/Umbrella_(children's_book)(1.0000014804076698)
http://dbpedia.org/resource/Viking_P

http://dbpedia.org/resource/Viking_Press  ---  http://dbpedia.org/ontology/publisher  ---  http://dbpedia.org/resource/Virtual_Light(1.0000014804076698)
http://dbpedia.org/resource/Viking_Press  ---  http://dbpedia.org/ontology/publisher  ---  http://dbpedia.org/resource/The_True_Story_of_the_Three_Little_Pigs(1.0000014804076698)
http://dbpedia.org/resource/Viking_Press  ---  http://dbpedia.org/ontology/publisher  ---  http://dbpedia.org/resource/The_Beginning_of_Infinity(1.0000014804076698)
http://dbpedia.org/resource/Viking_Press  ---  http://dbpedia.org/ontology/publisher  ---  http://dbpedia.org/resource/King_Solomon's_Carpet(1.0000014804076698)
http://dbpedia.org/resource/Viking_Press  ---  http://dbpedia.org/ontology/publisher  ---  http://dbpedia.org/resource/Where_They_Were_Missed(1.0000014804076698)
http://dbpedia.org/resource/Viking_Press  ---  http://dbpedia.org/ontology/publisher  ---  http://dbpedia.org/resource/The_Day_of_the_Jackal(1.0000014804076698)
http://dbpedia.org/

http://dbpedia.org/resource/Viking_Press  ---  http://dbpedia.org/ontology/publisher  ---  http://dbpedia.org/resource/The_Emperor_of_Ice-Cream_(novel)(1.0000014804076698)
http://dbpedia.org/resource/Viking_Press  ---  http://dbpedia.org/ontology/publisher  ---  http://dbpedia.org/resource/Ironweed_(novel)(1.0000014804076698)
http://dbpedia.org/resource/Viking_Press  ---  http://dbpedia.org/ontology/publisher  ---  http://dbpedia.org/resource/Dissolution_(Sansom_novel)(1.0000014804076698)
http://dbpedia.org/resource/Viking_Press  ---  http://dbpedia.org/ontology/publisher  ---  http://dbpedia.org/resource/Exterminator!(1.0000014804076698)
http://dbpedia.org/resource/Viking_Press  ---  http://dbpedia.org/ontology/publisher  ---  http://dbpedia.org/resource/Warlock_(Hall_novel)(1.0000014804076698)
http://dbpedia.org/resource/Viking_Press  ---  http://dbpedia.org/ontology/publisher  ---  http://dbpedia.org/resource/The_English_Gentleman(1.0000014804076698)
http://dbpedia.org/resource/Viki

http://dbpedia.org/resource/Viking_Press  ---  http://dbpedia.org/ontology/publisher  ---  http://dbpedia.org/resource/Syrup_(novel)(1.0000014804076698)
http://dbpedia.org/resource/Viking_Press  ---  http://dbpedia.org/ontology/publisher  ---  http://dbpedia.org/resource/Green_Monkey_Dreams(1.0000014804076698)
http://dbpedia.org/resource/Viking_Press  ---  http://dbpedia.org/ontology/publisher  ---  http://dbpedia.org/resource/Chaotic_(novella)(1.0000014804076698)
http://dbpedia.org/resource/Viking_Press  ---  http://dbpedia.org/ontology/publisher  ---  http://dbpedia.org/resource/The_Silent_Speaker(1.0000014804076698)
http://dbpedia.org/resource/Viking_Press  ---  http://dbpedia.org/ontology/publisher  ---  http://dbpedia.org/resource/How_to_Be_Good(1.0000014804076698)
http://dbpedia.org/resource/Viking_Press  ---  http://dbpedia.org/ontology/publisher  ---  http://dbpedia.org/resource/Mira,_Mirror(1.0000014804076698)
http://dbpedia.org/resource/Viking_Press  ---  http://dbpedia.org/o

http://dbpedia.org/resource/Jack_Kerouac  ---  http://dbpedia.org/ontology/artist  ---  http://dbpedia.org/resource/Blues_and_Haikus(0.45566913478574567)
http://dbpedia.org/resource/Jack_Kerouac  ---  http://dbpedia.org/ontology/artist  ---  http://dbpedia.org/resource/Poetry_for_the_Beat_Generation(0.45566913478574567)
http://dbpedia.org/resource/Jack_Kerouac  ---  http://dbpedia.org/property/quote  ---  The only people for me are the mad ones, the ones who are mad to live, mad to talk, mad to be saved, desirous of everything at the same time, the ones who never yawn or say a commonplace thing, but burn, burn, burn, like fabulous yellow Roman candles exploding like spiders across the stars, and in the middle, you see the blue center-light pop, and everybody goes ahh...(0.42689163211630826)
http://dbpedia.org/resource/Viking_Press  ---  http://dbpedia.org/property/imprints  ---  http://dbpedia.org/resource/Penguin_Books(0.41097423760036006)
http://dbpedia.org/resource/Viking_Press  ---

http://dbpedia.org/resource/Jack_Kerouac  ---  http://dbpedia.org/ontology/abstract  ---  Jack Kerouac (/ˈkɛruˌæk/ or /ˈkɛrəˌwæk/, born Jean-Louis Lebris de Kérouac; March 12, 1922 – October 21, 1969) was an American novelist and poet. He is considered a literary iconoclast and, alongside William S. Burroughs and Allen Ginsberg, a pioneer of the Beat Generation. Kerouac is recognized for his method of spontaneous prose. Thematically, his work covers topics such as Catholic spirituality, jazz, promiscuity, Buddhism, drugs, poverty, and travel. He became an underground celebrity and, with other beats, a progenitor of the hippie movement, although he remained antagonistic toward some of its politically radical elements. In 1969, aged 47, Kerouac died from internal bleeding due to long-term alcohol abuse. Since his death, Kerouac's literary prestige has grown, and several previously unseen works have been published. All of his books are in print today, including The Town and the City, On t

http://dbpedia.org/resource/Jack_Kerouac  ---  http://dbpedia.org/ontology/birthName  ---  Jean-Louis Lebris de Kérouac(0.2222222222222222)
http://dbpedia.org/resource/Viking_Press  ---  http://dbpedia.org/property/keypeople  ---  President-Brian Tart, Children's publisher Kenneth Wright(0.2222222222222222)
http://dbpedia.org/resource/Jack_Kerouac  ---  http://dbpedia.org/ontology/deathDate  ---  1969-10-21(0.2222222222222222)
http://dbpedia.org/resource/Jack_Kerouac  ---  http://dbpedia.org/ontology/birthDate  ---  1922-3-12(0.2222222222222222)
http://dbpedia.org/resource/Viking_Press  ---  http://xmlns.com/foaf/0.1/homepage  ---  http://penguin.com(0.21954058689691752)
http://dbpedia.org/resource/Viking_Press  ---  http://www.w3.org/ns/prov#wasDerivedFrom  ---  http://en.wikipedia.org/wiki/Viking_Press?oldid=738684384(0.21052631578947367)
http://dbpedia.org/resource/Jack_Kerouac  ---  http://www.w3.org/ns/prov#wasDerivedFrom  ---  http://en.wikipedia.org/wiki/Jack_Kerouac?oldid=74507

http://dbpedia.org/resource/Viking_Press  ---  http://www.w3.org/2000/01/rdf-schema#comment  ---  Viking Press è una casa editrice statunitense, fondata nel 1925 da Harold K. Guinzburg e George S. Oppenheimer con un logo rappresentante una nave vichinga.(0.12251229909837708)
http://dbpedia.org/resource/Viking_Press  ---  http://www.w3.org/2000/01/rdf-schema#comment  ---  Viking Press est une maison d'édition américaine fondée à New York en 1925 par Harold K. Guinzburg et George S. Oppenheim. Elle a été rachetée par Penguin Group en 1975. Le logo de la firme est un drakkar viking.(0.12251229909837708)
http://dbpedia.org/resource/Viking_Press  ---  http://www.w3.org/2000/01/rdf-schema#comment  ---  Viking Press é uma editora estadunidense agora propriedade da Penguin Random House. Foi fundada em Nova Iorque em 1 de março de 1925, por arold K. Guinzburg e George S. Oppenheim e, em seguida, adquirida pelo Penguin Group em 1975. O nome e logotipo da empresa—um navio viking desenhado por Roc

In [88]:
with open('215ques_correct.txt','r') as f:
    lines = f.readlines()
c = 0
lbs = []
current = []
while c < len(lines):
    line = lines[c].strip()
    if line.isnumeric():
        lbs.append(current)
        current = []
    else:
        #todo : check if correct
        if line.strip() not in current:
            current.append(line.strip())
    c += 1
lbs.pop(0)
lbs.append(['http://dbpedia.org/resource/Sparkling_wines','http://dbpedia.org/resource/France'])

# read keywords: original -> resultsKeywords.txt  Answer-> kwanswer.txt
with open('resultsKeywords.txt','r') as f:
    lines = f.readlines()
    
keywordsList = []
for x in lines:
    line = x.strip().split(',')
    i = 0
    while i < len(line):
        line[i] = line[i].strip()
        i += 1
    keywordsList.append(line)

In [89]:
with open('resultsKeywords.txt','r') as kw:
    kwlines = kw.readlines()
kwlist = []
for x in kwlines:
    x = x.strip()
    kwlist.append(x.split(','))

In [90]:
#print(lbs)
rslist = []
for x in lbs:
    current = []
    for y in x:
        y = y[y.rindex('resource')+9:]
        current.append(y)
    rslist.append(current)
print(rslist)

[['Battle_of_Gettysburg'], ['Elton_John'], ['Colombo_Lighthouse'], ['Abraham_Lincoln'], ['BBC_Wildlife_Specials'], ['Pulp_Fiction'], ['Heineken'], ['Operation_Overlord'], ['Chile_Route_68'], ['Slack_(software)'], ['Jacques_Chirac'], ['Bruce_Carver'], ['The_Storm_on_the_Sea_of_Galilee'], ['Weser'], ['FC_Porto'], ['Robert_F._Kennedy'], ['Viking_Press', 'Jack_Kerouac'], ['Amazon_Eve'], ['The_Scream'], ['The_Pillars_of_the_Earth'], ['James_Bond'], ['Ceres_(dwarf_planet)'], ['Brooklyn_Bridge'], [], [], ['Paris'], ['Ankara', 'Turkish_Ice_Hockey_First_League'], ['French_Polynesia'], ['Douglas_Hofstadter'], ['The_Lego_Movie'], ['Piccadilly'], ['Prince_William,_Duke_of_Cambridge', 'Prince_Harry'], ['Agent_Smith'], ['Turkmenistan'], ['Julia_Roberts', 'Richard_Gere'], ['Ganges'], ['Rachel_Stevens'], ['Barack_Obama', 'Democratic_Party_(United_States)'], ['Johann_Sebastian_Bach'], ['Jane_Fonda'], ['Mount_Everest'], ['Poland'], ['Palace_of_Westminster'], ['Batman_Begins', 'Christian_Bale'], ['Limeri

In [91]:
for x in range(215):
    kwl = kwlist[x]
    rsl = rslist[x]
    for j in rsl:
        currentK = ''
        currentS = 0
        for k in kwl:
            if levenshteinSim(k,j) > currentS:
                currentS = levenshteinSim(k,j)
                currentK = k
        if currentK != '':
            kwl.pop(kwl.index(currentK))

In [92]:
print(kwlist)

[[], ['musicals', ' music by'], [' lighthouse in Colombo'], ['U.S. president', ' wife'], ['host'], ['cost'], ['city'], ['when', ' commence'], ['city', ' ends'], ['developer'], ['married to'], [' die'], [' paint'], [' flow through', ' city'], ['How many seats'], [' daughter', ' married'], ['book', ' publish'], [' tall'], ['museum', ' Munch'], ['wrote', ' book'], [' married'], ['discoverer'], ['river', ' cross'], ['called', ' Frank The Tank'], ['largest country', ' world'], ['mayor'], ['coach'], ['mayor', ' capital'], [' award'], [' cost'], [' start'], [' same', ' parents'], ['played', ' Matrix'], ['languages'], ['film', ' play'], [' start', ' country'], ['year', ' born'], [], [' Bach'], [' marry', ' how often'], ['first', ' climb'], ['How many people'], [' Houses of Parliament'], [' starring'], [' country'], [' last', ' movie'], [' highest mountain'], [' alive'], ['create'], [' official language'], ['write', ' song'], ['actor', ' starring', ' movie', ' direct'], ['ingredients'], ['profe

In [93]:
with open('./newKeywordsList/resultsKeywords.txt','w') as nkw:
    for x in kwlist:
        cs = ''
        for y in x:
            cs += y.strip()+', '
        cs = cs[:-2]
        cs += '\n'
        nkw.write(cs)

In [115]:
with open('sparqlq.txt','r',encoding='utf-8') as f:
    lines = f.readlines()

with open('./newKeywordsList/onlykw.txt','w',encoding='utf-8') as fw:
    count = 0
    for x in lines:
        #print(str(count) + ':   ---')
        li = x.split()
        #print(li)
        for element in li:
            element = element.strip()
            if element.startswith('dbo:') and len(element) > 4:
                #print(element[4:])
                fw.write(', ' + element[4:])
            elif '/ontology/' in element and len(element) > 30:
                i = element.rindex('/')
                #print(element[i+1:-1])
                fw.write(', ' + element[i+1:-1])
        fw.write('\n')
        count += 1

In [116]:
with open('./newKeywordsList/onlykw.txt','r',encoding='utf-8') as fr:
    lines = fr.readlines()
with open('./newKeywordsList/onlykw.txt','w',encoding='utf-8') as fw:
    for x in lines:
        if len(x) < 5:
            fw.write('\n')
        fw.write(x[2:])

In [152]:
similarity('Musical','book')

0.564147082144386

In [161]:
for x in fetchTopK(eg2,5):
    print(x)

['musicBy', 'Musical']['musicBy', 'musicals']\http://dbpedia.org/resource/The_Lion_King_(musical)  ---  http://dbpedia.org/property/musicals  ---  http://dbpedia.org/resource/The_Lion_King_(franchise)(0.94, 0.94, 2) ||||| ['musicBy']['musicBy']\http://dbpedia.org/resource/Elton_John  ---  http://dbpedia.org/ontology/musicBy  ---  http://dbpedia.org/resource/The_Lion_King_(musical)(1.0, 1.0, 1) ||||| None
['musicBy', 'Musical']['musicBy', 'book']\http://dbpedia.org/resource/The_Lion_King_Jr_(musical)  ---  http://dbpedia.org/property/book  ---  Roger Allers and Irene Mecchi(0.56, 0.56, 2) ||||| ['musicBy']['musicBy']\http://dbpedia.org/resource/Elton_John  ---  http://dbpedia.org/ontology/musicBy  ---  http://dbpedia.org/resource/The_Lion_King_Jr_(musical)(1.0, 1.0, 1) ||||| None
['musicBy', 'Musical']['musicBy', 'productions']\http://dbpedia.org/resource/Lestat_(musical)  ---  http://dbpedia.org/property/productions  ---  2006(0.52, 0.52, 2) ||||| ['musicBy']['musicBy']\http://dbpedia.

In [160]:
eg2.sort()

In [155]:
str(eg2[8])

"['musicBy', 'Musical']['musicBy', 'book']\\http://dbpedia.org/resource/The_Lion_King_Jr_(musical)  ---  http://dbpedia.org/property/book  ---  Roger Allers and Irene Mecchi(0.56, 0.56, 2) ||||| ['musicBy']['musicBy']\\http://dbpedia.org/resource/Elton_John  ---  http://dbpedia.org/ontology/musicBy  ---  http://dbpedia.org/resource/The_Lion_King_Jr_(musical)(1.0, 1.0, 1) ||||| None"

In [38]:
lbs[24]

[]

In [54]:
queryString = "ASK WHERE{<http://dbpedia.org/resource/On_the_Road> rdf:type <http://dbpedia.org/ontology/Book>}"
sparql.setQuery(queryString)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()
print(results['boolean'])
#for result in results["results"]["bindings"]:
#    print(result)
    #return int(result['c']['value'])

True


In [44]:
ct = 0
while ct < len(lbs):
    if len(lbs[ct]) == 2:
        print(lbs[ct])
        print(ct)
    ct += 1

['http://dbpedia.org/resource/Viking_Press', 'http://dbpedia.org/resource/Jack_Kerouac']
16
['http://dbpedia.org/resource/Ankara', 'http://dbpedia.org/resource/Turkish_Ice_Hockey_First_League']
26
['http://dbpedia.org/resource/Prince_William,_Duke_of_Cambridge', 'http://dbpedia.org/resource/Prince_Harry']
31
['http://dbpedia.org/resource/Julia_Roberts', 'http://dbpedia.org/resource/Richard_Gere']
34
['http://dbpedia.org/resource/Barack_Obama', 'http://dbpedia.org/resource/Democratic_Party_(United_States)']
37
['http://dbpedia.org/resource/Batman_Begins', 'http://dbpedia.org/resource/Christian_Bale']
43
['http://dbpedia.org/resource/Neymar', 'http://dbpedia.org/resource/Real_Madrid_C.F.']
55
['http://dbpedia.org/resource/Computer_science', 'http://dbpedia.org/resource/Academy_Awards']
56
['http://dbpedia.org/resource/Secret_Intelligence_Service', 'http://dbpedia.org/resource/United_Kingdom']
58
['http://dbpedia.org/resource/Cuban_Missile_Crisis', 'http://dbpedia.org/resource/Bay_of_Pigs